# Consolidated Data Pipeline for Member Data - for testing data

Prepare by Tan Bee Hoon

16 Aug 2020

## Import Package

In [1]:
import random 
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import numpy as np 
from numpy import unique
from numpy import where

import pickle
from pandas import to_pickle
from pandas import read_pickle

from datetime import date

import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_colwidth', None)

## Set Global Variable

### Set input filenames

In [3]:
FILE_DIAG = "tbl_Booking_Diagnosis_test.csv"
FILE_PRESCRIPTION = "tbl_Booking_Prescription_test.csv"
FILE_SYMPTOMS = "tbl_Booking_Symptoms_test.csv"
FILE_BOOKING = "tbl_Booking_test.csv"
FILE_PATIENT = "tbl_Patients_test.csv"

###  Set for Testing or Training

In [4]:
# If its for test data (TESTING=1), to apply for model training, set min value as 0
# This is because for Testing hold out dataset purpose, will have to include previously bookings for existing members
TESTING = 1

### Set for Member or Booking Modeling

In [5]:
# For member model = 1, else for booking model = 0 
MODEL_TYPE_MEMBER = 1

### Set previous booking related source file (for Testing)

In [6]:
if TESTING == 1 and MODEL_TYPE_MEMBER == 1:
    FILE_DIAG_OLD = "tbl_Booking_Diagnosis.csv"
    FILE_PRESCRIPTION_OLD = "tbl_Booking_Prescription.csv"
    FILE_SYMPTOMS_OLD= "tbl_Booking_Symptoms.csv"
    FILE_BOOKING_OLD = "tbl_Booking.csv"

### Set minium count for feature considerations

In [7]:
if TESTING == 0: 
    MIN_VAL_SYM = 5
    MIN_VAL_DIAG = 10
elif TESTING == 1:
    MIN_VAL_SYM = 0
    MIN_VAL_DIAG = 0
else:
    print("Decide whether this data prep is for training or testing.")

## Helper Function

In [8]:
def count_unique(df):
    # List of categorical columns
    #cat_cols = df.select_dtypes('object').columns
    uniq_cols = df.columns
    
    list_count = []
    list_name = []
    
    for col in uniq_cols:
        # Formatting
        list_name.append(col)
        uniqueValues = df[col].nunique()
        list_count.append(uniqueValues)
        #print(str(col) + " has " + str(uniqueValues) + " values ")
        
    df_count = pd.DataFrame({"name" : list_name, "uniquecount" : list_count})
        
    return df_count

## Step1 - Intro Prep & Clean

* Extract list of patient
* Extract list of booking 
* Concatenate, Prep & Clean Booking Diagnosis
* Concatenate, Prep & Clean Booking Prescription
* Concatenate, Prep & Clean Booking Symptoms

### Extract list of patient

In [9]:
# Import Source Paitent Data
df_patient = pd.read_csv(FILE_PATIENT)
patient_list = df_patient["mem_id"].tolist()
print("List of patient: ", len(patient_list))

List of patient:  6774


### Extract list of booking

In [10]:
# For new booking list
df_booking = pd.read_csv(FILE_BOOKING)
print("BEFORE: ", "\n")
print(df_booking.info(), "\n")
df_booking = df_booking[df_booking['mem_id'].isin(patient_list)]
print("AFTER: ", "\n")
print(df_booking.info(), "\n")
booking_list = df_booking['booking_id'].tolist()
print("Number of booking: ", len(booking_list))

BEFORE:  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7725 entries, 0 to 7724
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 7725 non-null   object
 1   mem_id                     7725 non-null   object
 2   doctor_id                  7725 non-null   object
 3   consult_waived_reason      7725 non-null   object
 4   is_mc_issued               7725 non-null   bool  
 5   mc_days                    7725 non-null   int64 
 6   is_referral_letter_issued  7725 non-null   bool  
 7   booking_timestamp          7725 non-null   object
dtypes: bool(2), int64(1), object(5)
memory usage: 377.3+ KB
None 

AFTER:  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7725 entries, 0 to 7724
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 7725 no

In [11]:
# For old booking list (only required for member holdout dataset testing)
if TESTING == 1 and MODEL_TYPE_MEMBER == 1:
    df_booking_old = pd.read_csv(FILE_BOOKING_OLD)
    print(FILE_BOOKING_OLD)
    print("BEFORE: ","\n")
    print(df_booking_old.info(), "\n")
    df_booking_old = df_booking_old[df_booking_old['mem_id'].isin(patient_list)]
    print("AFTER: ", "\n")
    print(df_booking_old.info(), "\n")
    booking_old_list = df_booking_old["booking_id"].tolist()
    print("Number of booking from previous: ", len(booking_old_list))
    print("Number of member with previous booking: ",  len(df_booking_old['mem_id'].unique()))

tbl_Booking.csv
BEFORE:  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10221 entries, 0 to 10220
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
dtypes: bool(2), int64(1), object(5)
memory usage: 499.2+ KB
None 

AFTER:  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2233 entries, 1 to 10219
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id     

### Check overlapping booking ID between previous and new

In [12]:
if TESTING == 1 and MODEL_TYPE_MEMBER ==1 :
    set1 = set(booking_list)
    set2 = set(booking_old_list)
    newList = list(set1.intersection(set2))
    print("Intersection of the lists is:", newList)

Intersection of the lists is: []


if there is no intersection, it means the new dataset did not contain previous booking.

### Concatenate new and old booking list - for Testing

In [13]:
## Concatenate Booking ID
if TESTING == 1 and MODEL_TYPE_MEMBER == 1:
    booking_list = booking_list + booking_old_list

### Save Booking List 

In [14]:
# Export data 
open_file = open("step1_booking_list.pkl", "wb")
pickle.dump(booking_list, open_file)
open_file.close()

## Step1a - Prep & Clean Booking Diagnosis

### Import booking diagnosis data

In [15]:
# import new booking diagnosis
print("Input: ", FILE_DIAG, "\n") 
df_booking_diagnosis = pd.read_csv(FILE_DIAG)
df_booking_diagnosis = df_booking_diagnosis[df_booking_diagnosis['booking_id'].isin(booking_list)]
print(df_booking_diagnosis.info(), "\n") 
print("Shape of data: ", df_booking_diagnosis.shape)

Input:  tbl_Booking_Diagnosis_test.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 0 to 9199
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        9200 non-null   object
 1   MDDiagnosis_code  9200 non-null   object
 2   MDDiagnosis_name  9200 non-null   object
 3   chronic_group     9200 non-null   object
dtypes: object(4)
memory usage: 359.4+ KB
None 

Shape of data:  (9200, 4)


In [16]:
# import old booking diagnosis
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_DIAG_OLD, "\n") 
    df_booking_diagnosis_old = pd.read_csv(FILE_DIAG_OLD)
    df_booking_diagnosis_old = df_booking_diagnosis_old[df_booking_diagnosis_old['booking_id'].isin(booking_list)]
    print(df_booking_diagnosis_old.info(), "\n")
    print("Shape of data: ", df_booking_diagnosis_old.shape)


Input:  tbl_Booking_Diagnosis.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2874 entries, 1 to 65738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        2874 non-null   object
 1   MDDiagnosis_code  2874 non-null   object
 2   MDDiagnosis_name  2874 non-null   object
 3   chronic_group     2874 non-null   object
dtypes: object(4)
memory usage: 112.3+ KB
None 

Shape of data:  (2874, 4)


In [17]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1 :
    df_booking_diagnosis = pd.concat([df_booking_diagnosis, df_booking_diagnosis_old], axis=0)
    print("COMBINED: ")
    print(df_booking_diagnosis.info(), "\n")
    print("Shape of data: ", df_booking_diagnosis.shape)

COMBINED: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12074 entries, 0 to 65738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        12074 non-null  object
 1   MDDiagnosis_code  12074 non-null  object
 2   MDDiagnosis_name  12074 non-null  object
 3   chronic_group     12074 non-null  object
dtypes: object(4)
memory usage: 471.6+ KB
None 

Shape of data:  (12074, 4)


### Summary Statistics

In [18]:
print("Basic Data Summary")
df_booking_diagnosis.describe().transpose()

Basic Data Summary


,count,unique,top,freq
booking_id,12074,9958,89206016-CAA8-4E18-BD64-0EFAF312DAB9,6
MDDiagnosis_code,12074,577,J06.9,1111
MDDiagnosis_name,12074,578,"Acute upper respiratory infection, unspecified",1111
chronic_group,12074,11,-,10831


In [19]:
print("Unique value count for each column")
count_unique(df_booking_diagnosis)

Unique value count for each column


,name,uniquecount
0,booking_id,9958
1,MDDiagnosis_code,577
2,MDDiagnosis_name,578
3,chronic_group,11


### Missing Value Checks

In [20]:
print("number of null: ")
df_booking_diagnosis.isnull().sum()

number of null: 


booking_id          0
MDDiagnosis_code    0
MDDiagnosis_name    0
chronic_group       0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Basic Cleaning

#### Change text to all lower case

In [21]:
df_booking_diagnosis['MDDiagnosis_name'] = df_booking_diagnosis['MDDiagnosis_name'].str.lower()

#### Remove comma, replace with semi-colon 

In [22]:
df_booking_diagnosis.MDDiagnosis_name = df_booking_diagnosis.MDDiagnosis_name.str.replace(',', ';')

#### Check and remove duplicates

In [23]:
print("number of duplicate to drop: ", df_booking_diagnosis.duplicated().sum())

number of duplicate to drop:  1


In [24]:
# to drop duplicates 
if df_booking_diagnosis.duplicated().sum() > 0:
    df_booking_diagnosis = df_booking_diagnosis.drop_duplicates()

### Update Data Types

#### Change data types for categorical data

In [25]:
df_booking_diagnosis['MDDiagnosis_code'] = df_booking_diagnosis['MDDiagnosis_code'].astype("category")

In [26]:
df_booking_diagnosis['MDDiagnosis_name'] = df_booking_diagnosis['MDDiagnosis_name'].astype("category")

In [27]:
df_booking_diagnosis['chronic_group'] = df_booking_diagnosis['chronic_group'].astype("category")

In [28]:
df_booking_diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12073 entries, 0 to 65738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   booking_id        12073 non-null  object  
 1   MDDiagnosis_code  12073 non-null  category
 2   MDDiagnosis_name  12073 non-null  category
 3   chronic_group     12073 non-null  category
dtypes: category(3), object(1)
memory usage: 289.3+ KB


### Check OntoOne Mapping

In [29]:
g = df_booking_diagnosis[['MDDiagnosis_code', 'MDDiagnosis_name']].groupby('MDDiagnosis_name')
counts = g.transform(lambda x: len(x.unique()))
df_booking_diagnosis["count_map"] = counts

In [30]:
print("These are the records with more than one mapping for name vs code, please verify:")

These are the records with more than one mapping for name vs code, please verify:


In [31]:
pd.set_option('display.max_colwidth', None)
df_booking_diagnosis.loc[df_booking_diagnosis["count_map"]>1,:]

,booking_id,MDDiagnosis_code,MDDiagnosis_name,chronic_group,count_map


<span class="mark">IMPORTANT: 
Please verify data if above shows that there is duplicate mapping.</span> 

#### Remove Redundant Mapping

In [32]:
df_booking_diagnosis = df_booking_diagnosis.drop_duplicates(subset=["booking_id", "MDDiagnosis_name"])

In [33]:
df_booking_diagnosis.drop(["count_map"], inplace = True, axis = 1)

In [34]:
df_booking_diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12073 entries, 0 to 65738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   booking_id        12073 non-null  object  
 1   MDDiagnosis_code  12073 non-null  category
 2   MDDiagnosis_name  12073 non-null  category
 3   chronic_group     12073 non-null  category
dtypes: category(3), object(1)
memory usage: 289.3+ KB


In [35]:
print("Unique value count for each column")
count_unique(df_booking_diagnosis)

Unique value count for each column


,name,uniquecount
0,booking_id,9958
1,MDDiagnosis_code,577
2,MDDiagnosis_name,577
3,chronic_group,11


### Data Aggregation

#### Check Unique Diagnosis

In [36]:
print(df_booking_diagnosis["MDDiagnosis_name"].nunique())

577


In [37]:
print(df_booking_diagnosis['MDDiagnosis_name'].value_counts().head(20))

acute upper respiratory infection; unspecified                               1111
headache                                                                     1058
gastroenteritis and colitis of unspecified origin                            1009
covid-19 vaccines causing adverse effects in therapeutic use                  842
other complications following immunization; not elsewhere classified          512
dysmenorrhoea; unspecified                                                    432
rash and other nonspecific skin eruption                                      292
myalgia                                                                       244
muscle strain                                                                 243
person consulting for explanation of investigation findings                   228
covid-19 vaccines causing adverse effects in therapeutic use; unspecified     202
fever; unspecified                                                            202
gastritis; unspe

In [38]:
print(df_booking_diagnosis['MDDiagnosis_name'].value_counts(normalize=True).head(20))

acute upper respiratory infection; unspecified                               0.092024
headache                                                                     0.087634
gastroenteritis and colitis of unspecified origin                            0.083575
covid-19 vaccines causing adverse effects in therapeutic use                 0.069742
other complications following immunization; not elsewhere classified         0.042409
dysmenorrhoea; unspecified                                                   0.035782
rash and other nonspecific skin eruption                                     0.024186
myalgia                                                                      0.020210
muscle strain                                                                0.020128
person consulting for explanation of investigation findings                  0.018885
covid-19 vaccines causing adverse effects in therapeutic use; unspecified    0.016732
fever; unspecified                                    

#### Check Unique Chronic

In [39]:
print(df_booking_diagnosis["chronic_group"].nunique())

11


In [40]:
print(df_booking_diagnosis["chronic_group"].value_counts())

-                                    10830
Vasomotor and allergic rhinitis        308
Hypertension                           158
Hyperlipidaemia                        144
Atopic dermatitis                      133
Gastro-oesophageal reflux disease      131
Migraine                               124
Gout                                    90
Asthma                                  84
Thyroid                                 37
Diabetes                                34
Name: chronic_group, dtype: int64


In [41]:
print(df_booking_diagnosis["chronic_group"].value_counts(normalize=True)*100)

-                                    89.704299
Vasomotor and allergic rhinitis       2.551147
Hypertension                          1.308705
Hyperlipidaemia                       1.192744
Atopic dermatitis                     1.101632
Gastro-oesophageal reflux disease     1.085066
Migraine                              1.027085
Gout                                  0.745465
Asthma                                0.695767
Thyroid                               0.306469
Diabetes                              0.281620
Name: chronic_group, dtype: float64


#### Rollup

The purpose here is to roll up all the diagnose of the same booking into one roll, while retaining all diagnosis info, by converting the information into a list.

In [42]:
# get a dict of unique ID from booking  
booking_ID_list = set(df_booking_diagnosis["booking_id"])

In [43]:
diag_dup_list = pd.DataFrame(columns=["booking_id","diagnosis", "count_diagnosis"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_diagnosis[df_booking_diagnosis["booking_id"] == x]
    diag_list_tmp = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDDiagnosis_name"] is not np.nan:
            if (row["MDDiagnosis_name"] not in diag_list_tmp):
                diag_list_tmp.append(row["MDDiagnosis_name"])
             
    diag_dup_list.at[x, "booking_id"] = x
    diag_dup_list.at[x, "diagnosis"] = diag_list_tmp
    diag_dup_list.at[x, "count_diagnosis"] = int(len(diag_list_tmp))

In [44]:
diagcode_dup_list = pd.DataFrame(columns=["booking_id","diagnosis_code", "count_diagnosis_code"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_diagnosis[df_booking_diagnosis["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDDiagnosis_code"] is not np.nan:
            if (row["MDDiagnosis_code"] not in tmp_list):
                tmp_list.append(row["MDDiagnosis_code"])
             
    diagcode_dup_list.at[x, "booking_id"] = x
    diagcode_dup_list.at[x, "diagnosis_code"] = tmp_list
    diagcode_dup_list.at[x, "count_diagnosis_code"] = int(len(tmp_list))

In [45]:
chronic_dup_list = pd.DataFrame(columns=["booking_id","chronic", "count_chronic"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_diagnosis[df_booking_diagnosis["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["chronic_group"] is not np.nan and row["chronic_group"] is not "-":
            if (row["chronic_group"] not in tmp_list):
                tmp_list.append(row["chronic_group"])
        
    chronic_dup_list.at[x, "booking_id"] = x
    chronic_dup_list.at[x, "chronic"] = tmp_list
    chronic_dup_list.at[x, "count_chronic"] = int(len(tmp_list))

In [46]:
# Perform an inner join between the temp diagnosis name and code tables
combine_data_diag = pd.merge(diagcode_dup_list, diag_dup_list, how="inner", on=["booking_id"])

In [47]:
# Perform an inner join between the chronic temp table & above
combine_data_diag_chronic = pd.merge(combine_data_diag, chronic_dup_list, how="inner", on=["booking_id"])

### Finalise Prep 

In [48]:
final_df_booking_diagnosis = combine_data_diag_chronic[["booking_id","diagnosis_code","diagnosis","chronic", 
                                      "count_diagnosis", "count_chronic"]]

In [49]:
final_df_booking_diagnosis.columns = ['booking_id', 'diagnosis_code_list', 'diagnosis_list', 'chronic_list',
       'count_diagnosis', 'count_chronic']

In [50]:
final_df_booking_diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           9958 non-null   object
 1   diagnosis_code_list  9958 non-null   object
 2   diagnosis_list       9958 non-null   object
 3   chronic_list         9958 non-null   object
 4   count_diagnosis      9958 non-null   object
 5   count_chronic        9958 non-null   object
dtypes: object(6)
memory usage: 544.6+ KB


In [51]:
final_df_booking_diagnosis.head(5)

,booking_id,diagnosis_code_list,diagnosis_list,chronic_list,count_diagnosis,count_chronic
0,65B1CE93-A2A3-44F4-942B-5A799CAA3D22,[R11],[nausea and vomiting],[],1,0
1,1AAF3D3C-5DE4-466A-9F52-F673475D8EC5,[U12],[covid-19 vaccines causing adverse effects in therapeutic use],[],1,0
2,57164B4A-6871-479A-85E2-A7BED21B42B3,[A09],[other gastroenteritis and colitis of infectious and unspecified origin],[],1,0
3,DC93593E-DCE1-4527-970A-118BF107BC2E,[R21],[rash and other nonspecific skin eruption],[],1,0
4,19630CC6-8E9E-42DB-87D2-6674B91427EE,[Z71.2],[person consulting for explanation of investigation findings],[],1,0


### Save intermediate Output 

In [52]:
# Export data 
final_df_booking_diagnosis.to_pickle("step1_booking_diagnosis.pkl")

<span class="burk">**COMPLETE STEP1 FOR BOOKING DIAGNOSIS**</span>

## Step1b - Prep & Clean Booking Prescription

### Import booking Prescription data

In [53]:
# import new booking prescription
print("Input: ", FILE_PRESCRIPTION, "\n") 
df_booking_presciption = pd.read_csv(FILE_PRESCRIPTION)
df_booking_presciption = df_booking_presciption[df_booking_presciption['booking_id'].isin(booking_list)]
print(df_booking_presciption.info(), "\n") 
print("Shape of data: ", df_booking_presciption.shape)

Input:  tbl_Booking_Prescription_test.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14926 entries, 0 to 14925
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   booking_id               14926 non-null  object 
 1   MDMedication_ID          14926 non-null  object 
 2   MDMedication_name        14926 non-null  object 
 3   is_fulfilled_externally  14926 non-null  bool   
 4   prescription_quantity    14926 non-null  float64
 5   prescription_status      14926 non-null  object 
dtypes: bool(1), float64(1), object(4)
memory usage: 714.2+ KB
None 

Shape of data:  (14926, 6)


In [54]:
# import old booking prescription
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_PRESCRIPTION_OLD, "\n") 
    df_booking_presciption_old = pd.read_csv(FILE_PRESCRIPTION_OLD)
    df_booking_presciption_old = df_booking_presciption_old[df_booking_presciption_old['booking_id'].isin(booking_list)]
    print(df_booking_presciption_old.info(), "\n")
    print("Shape of data: ", df_booking_presciption_old.shape)

Input:  tbl_Booking_Prescription.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4436 entries, 4 to 20238
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   booking_id               4436 non-null   object
 1   MDMedication_ID          4436 non-null   object
 2   MDMedication_name        4436 non-null   object
 3   is_fulfilled_externally  4436 non-null   bool  
 4   prescription_quantity    4436 non-null   int64 
 5   prescription_status      4436 non-null   object
dtypes: bool(1), int64(1), object(4)
memory usage: 212.3+ KB
None 

Shape of data:  (4436, 6)


In [55]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1:
    df_booking_presciption = pd.concat([df_booking_presciption, df_booking_presciption_old], axis=0)
    print("COMBINED: ")
    print(df_booking_presciption.info(), "\n")
    print("Shape of data: ", df_booking_presciption.shape)

COMBINED: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19362 entries, 0 to 20238
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   booking_id               19362 non-null  object 
 1   MDMedication_ID          19362 non-null  object 
 2   MDMedication_name        19362 non-null  object 
 3   is_fulfilled_externally  19362 non-null  bool   
 4   prescription_quantity    19362 non-null  float64
 5   prescription_status      19362 non-null  object 
dtypes: bool(1), float64(1), object(4)
memory usage: 926.5+ KB
None 

Shape of data:  (19362, 6)


### Summary Statistics

In [56]:
print("Basic Data Summary")
df_booking_presciption.describe().transpose()

Basic Data Summary


,count,mean,std,min,25%,50%,75%,max
prescription_quantity,19362.0,15.837104,16.405406,1.0,6.0,10.0,20.0,500.0


In [57]:
print("Unique value count for each column")
count_unique(df_booking_presciption)

Unique value count for each column


,name,uniquecount
0,booking_id,7606
1,MDMedication_ID,330
2,MDMedication_name,335
3,is_fulfilled_externally,2
4,prescription_quantity,50
5,prescription_status,2


### Missing Value Checks

In [58]:
print("number of null: ")
df_booking_presciption.isnull().sum()

number of null: 


booking_id                 0
MDMedication_ID            0
MDMedication_name          0
is_fulfilled_externally    0
prescription_quantity      0
prescription_status        0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Basic Cleaning

#### Remove comma, replace with semi-colon 

In [59]:
df_booking_presciption.MDMedication_name = df_booking_presciption.MDMedication_name.str.replace(',', ';')

#### Check and remove duplicates

In [60]:
print("number of duplicate to drop: ", df_booking_presciption.duplicated().sum())

number of duplicate to drop:  11


In [61]:
if df_booking_presciption.duplicated().sum() > 0:
    df_booking_presciption = df_booking_presciption.drop_duplicates()

### Update Data Types

#### Change data types for categorical data

In [62]:
df_booking_presciption['MDMedication_ID'] = df_booking_presciption['MDMedication_ID'].astype("category")

In [63]:
df_booking_presciption['MDMedication_name'] = df_booking_presciption['MDMedication_name'].astype("category")

In [64]:
df_booking_presciption['prescription_status'] = df_booking_presciption['prescription_status'].astype("category")

In [65]:
df_booking_presciption.is_fulfilled_externally = df_booking_presciption.is_fulfilled_externally.replace({True: "True", False: "False"})
df_booking_presciption.is_fulfilled_externally = df_booking_presciption.is_fulfilled_externally.astype("category")

In [66]:
df_booking_presciption.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 20238
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   booking_id               19351 non-null  object  
 1   MDMedication_ID          19351 non-null  category
 2   MDMedication_name        19351 non-null  category
 3   is_fulfilled_externally  19351 non-null  category
 4   prescription_quantity    19351 non-null  float64 
 5   prescription_status      19351 non-null  category
dtypes: category(4), float64(1), object(1)
memory usage: 588.6+ KB


### Check OntoOne Mapping

In [67]:
g = df_booking_presciption[['MDMedication_ID', 'MDMedication_name']].groupby('MDMedication_name')
counts = g.transform(lambda x: len(x.unique()))

In [68]:
df_booking_presciption["count_map"] = counts

In [69]:
print("These are the records with more than one mapping for name vs code, please verify:")

These are the records with more than one mapping for name vs code, please verify:


In [70]:
pd.set_option('display.max_colwidth', None)
df_booking_presciption.loc[df_booking_presciption["count_map"]>1,:]

,booking_id,MDMedication_ID,MDMedication_name,is_fulfilled_externally,prescription_quantity,prescription_status,count_map


<span class="mark">IMPORTANT: 
Please verify data if above shows that there is duplicate mapping.</span> 

#### Remove Redundant field

In [71]:
df_booking_presciption.drop(["count_map"], inplace = True, axis = 1)

In [72]:
print("Unique value count for each column")
count_unique(df_booking_presciption)

Unique value count for each column


,name,uniquecount
0,booking_id,7606
1,MDMedication_ID,330
2,MDMedication_name,335
3,is_fulfilled_externally,2
4,prescription_quantity,50
5,prescription_status,2


### Data Aggregation

#### Check Unique Medication ID

In [73]:
print(df_booking_presciption["MDMedication_ID"].nunique())

330


#### Check Unique Medication Name

In [74]:
print(df_booking_presciption["MDMedication_name"].nunique())

335


#### Check Unique prescription_quantity

In [75]:
print(df_booking_presciption["prescription_quantity"].nunique())

50


#### Roll up

The purpose here is to roll up all the diagnose of the same booking into one roll, while retaining all diagnosis info, by converting the information into a list.

In [76]:
# get a dict of unique ID from booking  
booking_ID_list = set(df_booking_presciption["booking_id"])

In [77]:
# initiate df to keep repeated medication for prescription
col_list = df_booking_presciption.columns.tolist()
df_repeat = pd.DataFrame(columns= col_list) 

In [78]:
medID_dup_list = pd.DataFrame(columns=["booking_id","medicationID_list", "count_medication", "count_repeatmed"])
# to get a list of medication ID for each booking
count_repeat_overall = 0
for x in booking_ID_list:
    count_repeat_booking = 1
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    for index, row in temp_df.iterrows():
        if row["MDMedication_ID"] is not np.nan:
            if (row["MDMedication_ID"] not in temp_list):
                temp_list.append(row["MDMedication_ID"])
            else:
                count_repeat_booking += 1
                count_repeat_overall += 1
                df_repeat = df_repeat.append(row)
                #print("Repeated: ", str(row), "\n")
             
    medID_dup_list.at[x, "booking_id"] = x
    medID_dup_list.at[x, "medicationID_list"] = temp_list
    medID_dup_list.at[x, "count_medication"] = int(len(temp_list))
    medID_dup_list.at[x, "count_repeatmed"] = int(count_repeat_booking)
print("TOTAL REPEATED MED ROWS: ", str(count_repeat_overall))

TOTAL REPEATED MED ROWS:  24


In [79]:
medName_dup_list = pd.DataFrame(columns=["booking_id","medicationName_list", "count_medName"])
# to get a list of medication ID for each booking
for x in booking_ID_list:
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDMedication_name"] is not np.nan:
            if (row["MDMedication_name"] not in temp_list):
                temp_list.append(row["MDMedication_name"])
             
    medName_dup_list.at[x, "booking_id"] = x
    medName_dup_list.at[x, "medicationName_list"] = temp_list
    medName_dup_list.at[x, "count_medName"] = int(len(temp_list))

In [80]:
medquantity_dup_list = pd.DataFrame(columns=["booking_id","medQuantity_list", "count_medQuantity"])
# to get a list of medication ID for each booking
for x in booking_ID_list:
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    check_list = []
    for index, row in temp_df.iterrows():
        if row["prescription_quantity"] is not np.nan:
            if (row["MDMedication_name"] not in check_list):
                check_list.append(row["MDMedication_name"])
                temp_list.append(row["prescription_quantity"])
    medquantity_dup_list.at[x, "booking_id"] = x
    medquantity_dup_list.at[x, "medQuantity_list"] = temp_list
    medquantity_dup_list.at[x, "count_medQuantity"] = int(len(temp_list))

In [81]:
medAndQuan_dup_list = pd.DataFrame(columns=["booking_id","medAndQuan_list", "medNQuanCount"])
# to get a list of medication ID for each booking
for x in booking_ID_list:
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    check_list = []
    for index, row in temp_df.iterrows():
        if row["prescription_quantity"] is not np.nan:
            if (row["MDMedication_name"] not in check_list):
                check_list.append(row["MDMedication_name"])
                temp_list.append(str(row["MDMedication_name"]) + "|||" + str(row["prescription_quantity"]))
             
    medAndQuan_dup_list.at[x, "booking_id"] = x
    medAndQuan_dup_list.at[x, "medAndQuan_list"] = temp_list
    medAndQuan_dup_list.at[x, "medNQuanCount"] = int(len(temp_list))

In [82]:
# Perform an inner join between the prescription name and code temp tables
combine_data_med1 = pd.merge(medID_dup_list, medName_dup_list, how="inner", on=["booking_id"])

In [83]:
# Perform an inner join between the above and quantity temp tables
combine_data_med2 = pd.merge(combine_data_med1, medquantity_dup_list, how="inner", on=["booking_id"])

In [84]:
# Perform an inner join between the above and medication&quantity pairs temp tables
combine_data_med3 = pd.merge(combine_data_med2, medAndQuan_dup_list, how="inner", on=["booking_id"])

#### Finalise Prep

In [85]:
col_list = ["booking_id","medicationID_list","medicationName_list","medQuantity_list", "medAndQuan_list", 
            "count_medication", "count_repeatmed"]

In [86]:
final_df_booking_presciption = combine_data_med3[col_list]

In [87]:
final_df_booking_presciption.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7606 entries, 0 to 7605
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           7606 non-null   object
 1   medicationID_list    7606 non-null   object
 2   medicationName_list  7606 non-null   object
 3   medQuantity_list     7606 non-null   object
 4   medAndQuan_list      7606 non-null   object
 5   count_medication     7606 non-null   object
 6   count_repeatmed      7606 non-null   object
dtypes: object(7)
memory usage: 475.4+ KB


In [88]:
final_df_booking_presciption.head(3)

,booking_id,medicationID_list,medicationName_list,medQuantity_list,medAndQuan_list,count_medication,count_repeatmed
0,DC93593E-DCE1-4527-970A-118BF107BC2E,[1A190BBB-8C24-4E30-8125-8B25BFD3FE42],[* Fusidic Acid 2% /Betamethasone Valerate 0.1% cream (5g) (Fucicort)],[1.0],[* Fusidic Acid 2% /Betamethasone Valerate 0.1% cream (5g) (Fucicort)|||1.0],1,1
1,8E858016-AE8C-415A-8107-B796BDD69DA8,"[C56B89D0-BA5A-4B60-A156-0CB20FBE56E0, 472C187F-FD51-45B6-9C4A-2F75BAF63CAE, A9AC45A0-5E2E-4FEF-B244-3DFCAD178132, 60CC1143-AFF5-4690-A9A3-9414A01037E6]","[Acetylcysteine 600mg Effervescent tab (Sandoz), Dextromethorphan 15mg/5ml syrup (90ml) (Tussidex), MAC Dual Action Lozenges, Cetirizine 10mg tab (Adezio)]","[10.0, 1.0, 12.0, 10.0]","[Acetylcysteine 600mg Effervescent tab (Sandoz)|||10.0, Dextromethorphan 15mg/5ml syrup (90ml) (Tussidex)|||1.0, MAC Dual Action Lozenges|||12.0, Cetirizine 10mg tab (Adezio)|||10.0]",4,1
2,1FA8669F-F9E3-4C9F-B4F4-D5FAA562C87C,[B82D9685-75DA-4AC0-8DC5-38C7DF3B1E59],[Metronidazole 200mg tab],[30.0],[Metronidazole 200mg tab|||30.0],1,1


#### Save intermediate Output

In [89]:
# Export data 
final_df_booking_presciption.to_pickle("step1_booking_prescription.pkl")

<span class="burk">**COMPLETE STEP1 FOR BOOKING PRESCRIPTION**</span>

## Step1c - Prep & Clean Booking Symptoms

### Import booking symptoms data

In [90]:
# import new booking symptoms
print("Input: ", FILE_SYMPTOMS, "\n") 
df_booking_symptoms = pd.read_csv(FILE_SYMPTOMS)
df_booking_symptoms = df_booking_symptoms[df_booking_symptoms['booking_id'].isin(booking_list)]
print(df_booking_symptoms.info(), "\n") 
print("Shape of data: ", df_booking_symptoms.shape)

Input:  tbl_Booking_Symptoms_test.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15263 entries, 0 to 15262
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   booking_id      15263 non-null  object
 1   MDSymptom_id    15263 non-null  object
 2   MDSymptom_name  15263 non-null  object
dtypes: object(3)
memory usage: 477.0+ KB
None 

Shape of data:  (15263, 3)


In [91]:
# import old booking symptoms
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_SYMPTOMS_OLD, "\n") 
    df_booking_symptoms_old = pd.read_csv(FILE_SYMPTOMS_OLD)
    df_booking_symptoms_old = df_booking_symptoms_old[df_booking_symptoms_old['booking_id'].isin(booking_list)]
    print(df_booking_symptoms_old.info(), "\n")
    print("Shape of data: ", df_booking_symptoms_old.shape)

Input:  tbl_Booking_Symptoms.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3826 entries, 27 to 18089
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   booking_id      3826 non-null   object
 1   MDSymptom_id    3826 non-null   object
 2   MDSymptom_name  3826 non-null   object
dtypes: object(3)
memory usage: 119.6+ KB
None 

Shape of data:  (3826, 3)


In [92]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1:
    df_booking_symptoms = pd.concat([df_booking_symptoms, df_booking_symptoms_old], axis=0)
    print("COMBINED: ")
    print(df_booking_symptoms.info(), "\n")
    print("Shape of data: ", df_booking_symptoms.shape)

COMBINED: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19089 entries, 0 to 18089
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   booking_id      19089 non-null  object
 1   MDSymptom_id    19089 non-null  object
 2   MDSymptom_name  19089 non-null  object
dtypes: object(3)
memory usage: 596.5+ KB
None 

Shape of data:  (19089, 3)


### Summary Statistics

In [93]:
print("Basic Data Summary")
df_booking_symptoms.describe().transpose()

Basic Data Summary


,count,unique,top,freq
booking_id,19089,8870,B4D8998D-E528-4725-91C6-F9F6E54A3DC0,10
MDSymptom_id,19089,20,A518BD9F-95E0-411F-B058-F4DEA6AD1884,3506
MDSymptom_name,19089,20,others,3506


In [94]:
print("Unique value count for each column")
count_unique(df_booking_symptoms)

Unique value count for each column


,name,uniquecount
0,booking_id,8870
1,MDSymptom_id,20
2,MDSymptom_name,20


### Missing Value Checks

In [95]:
print("number of null: ")
df_booking_symptoms.isnull().sum()

number of null: 


booking_id        0
MDSymptom_id      0
MDSymptom_name    0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Basic Cleaning

#### Change text to all lower case

In [96]:
df_booking_symptoms['MDSymptom_name'] = df_booking_symptoms['MDSymptom_name'].str.lower()

#### Remove comma, replace with semi-colon 

In [97]:
df_booking_symptoms.MDSymptom_name = df_booking_symptoms.MDSymptom_name.str.replace(',', ';')

#### Check and remove duplicates

In [98]:
print("number of duplicate to drop: ", df_booking_symptoms.duplicated().sum())

number of duplicate to drop:  0


In [99]:
# to drop duplicates 
if df_booking_symptoms.duplicated().sum() > 0:
    df_booking_symptoms = df_booking_symptoms.drop_duplicates()

### Update Data Types

#### Change data types for categorical data

In [100]:
df_booking_symptoms['MDSymptom_id'] = df_booking_symptoms['MDSymptom_id'].astype("category")

In [101]:
df_booking_symptoms['MDSymptom_name'] = df_booking_symptoms['MDSymptom_name'].astype("category")

In [102]:
df_booking_symptoms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19089 entries, 0 to 18089
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   booking_id      19089 non-null  object  
 1   MDSymptom_id    19089 non-null  category
 2   MDSymptom_name  19089 non-null  category
dtypes: category(2), object(1)
memory usage: 336.9+ KB


In [103]:
print("List of Chronic Disease:")
df_booking_symptoms['MDSymptom_name'].unique().tolist()

List of Chronic Disease:


['vomiting',
 'diarrhoea',
 'others',
 'sore throat',
 'flu',
 'cough',
 'cold',
 'fever',
 'headache',
 'giddiness',
 'cold sores',
 'rash',
 'medical referral',
 'chronic medicine top-up',
 'discuss lab results',
 'chronic illness management',
 'ulcer under tongue',
 'skin growth on sole (foot)',
 'runny nose',
 'vertigo']

### Check OntoOne Mapping

#### Check by ID

In [104]:
g = df_booking_symptoms[['MDSymptom_id', 'MDSymptom_name']].groupby('MDSymptom_id')
counts = g.transform(lambda x: len(x.unique()))
(counts>1).sum()

MDSymptom_name    0
dtype: int64

#### Check by Name

In [105]:
g2 = df_booking_symptoms[['MDSymptom_id', 'MDSymptom_name']].groupby('MDSymptom_name')
counts2 = g2.transform(lambda x: len(x.unique()))
(counts2>1).sum()

MDSymptom_id    0
dtype: int64

<span class="mark">IMPORTANT: 
Please verify data if above shows that there is duplicate mapping.</span> 

### Data Aggregation

#### Check unique symptoms ID

In [106]:
print(df_booking_symptoms["MDSymptom_id"].nunique())

20


#### Check unique symptoms Name

In [107]:
print(df_booking_symptoms["MDSymptom_name"].nunique())

20


In [108]:
print(df_booking_symptoms["MDSymptom_name"].value_counts())

others                        3506
headache                      2355
giddiness                     2203
sore throat                   1404
cough                         1380
flu                           1376
cold                          1330
diarrhoea                     1291
vomiting                      1245
fever                         1113
rash                           448
cold sores                     430
discuss lab results            335
medical referral               318
chronic medicine top-up        311
chronic illness management      40
runny nose                       1
skin growth on sole (foot)       1
ulcer under tongue               1
vertigo                          1
Name: MDSymptom_name, dtype: int64


In [109]:
print(df_booking_symptoms["MDSymptom_name"].value_counts(normalize=True)*100)

others                        18.366599
headache                      12.336948
giddiness                     11.540678
sore throat                    7.355021
cough                          7.229294
flu                            7.208340
cold                           6.967363
diarrhoea                      6.763057
vomiting                       6.522081
fever                          5.830583
rash                           2.346901
cold sores                     2.252606
discuss lab results            1.754937
medical referral               1.665881
chronic medicine top-up        1.629211
chronic illness management     0.209545
runny nose                     0.005239
skin growth on sole (foot)     0.005239
ulcer under tongue             0.005239
vertigo                        0.005239
Name: MDSymptom_name, dtype: float64


#### Roll up

In [110]:
# get a dict of unique ID from booking  
booking_ID_list = set(df_booking_symptoms["booking_id"])

In [111]:
smyID_dup_list = pd.DataFrame(columns=["booking_id","SymptomID_list", "count_SymID"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_symptoms[df_booking_symptoms["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDSymptom_id"] is not np.nan:
            if (row["MDSymptom_id"] not in tmp_list):
                tmp_list.append(row["MDSymptom_id"])
             
    smyID_dup_list.at[x, "booking_id"] = x
    smyID_dup_list.at[x, "SymptomID_list"] = tmp_list
    smyID_dup_list.at[x, "count_SymID"] = int(len(tmp_list))

In [112]:
smyName_dup_list = pd.DataFrame(columns=["booking_id","SymptomName_list", "count_SymName"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_symptoms[df_booking_symptoms["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDSymptom_name"] is not np.nan:
            if (row["MDSymptom_name"] not in tmp_list):
                tmp_list.append(row["MDSymptom_name"])
             
    smyName_dup_list.at[x, "booking_id"] = x
    smyName_dup_list.at[x, "SymptomName_list"] = tmp_list
    smyName_dup_list.at[x, "count_SymName"] = int(len(tmp_list))

### Finalise Prep

In [113]:
# Perform an inner join between the diagnosis name and code temp tables
combine_data_symp = pd.merge(smyID_dup_list, smyName_dup_list, how="inner", on=["booking_id"])

In [114]:
final_df_booking_symptoms = combine_data_symp[["booking_id","SymptomID_list","SymptomName_list",
                              "count_SymID"]]

In [115]:
final_df_booking_symptoms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8870 entries, 0 to 8869
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        8870 non-null   object
 1   SymptomID_list    8870 non-null   object
 2   SymptomName_list  8870 non-null   object
 3   count_SymID       8870 non-null   object
dtypes: object(4)
memory usage: 346.5+ KB


In [116]:
final_df_booking_symptoms.shape

(8870, 4)

In [117]:
final_df_booking_symptoms.head(3)

,booking_id,SymptomID_list,SymptomName_list,count_SymID
0,65B1CE93-A2A3-44F4-942B-5A799CAA3D22,"[C3AF5B1F-BD74-4700-A1E7-1958B32C80BD, B8F546F4-9C2A-4E31-8ED5-DB3187FA4351]","[vomiting, diarrhoea]",2
1,1AAF3D3C-5DE4-466A-9F52-F673475D8EC5,[A518BD9F-95E0-411F-B058-F4DEA6AD1884],[others],1
2,57164B4A-6871-479A-85E2-A7BED21B42B3,"[C3AF5B1F-BD74-4700-A1E7-1958B32C80BD, 6E7F72BF-3C7B-4F0D-B1D2-5E9FB9B5CE9B, B8F546F4-9C2A-4E31-8ED5-DB3187FA4351, 1B6BFF99-1553-487A-A85E-EBF552D50D23, A518BD9F-95E0-411F-B058-F4DEA6AD1884]","[vomiting, headache, diarrhoea, giddiness, others]",5


### Save intermediate Ouput

In [118]:
# Export data 
final_df_booking_symptoms.to_pickle("step1_booking_symptoms.pkl")

<span class="burk">**COMPLETE STEP1 FOR BOOKING SYMPTOMS**</span>

<span class="burk">COMPLETE ALL STEP1</span>

## Step2 - Merge Data Tables 

Merge Strategy:

1. Combine Booking and Patient to become the Base
2. Combine Base + Diagnosis(Inner) + Prescription(Outer) + Symptoms(Outer)

Note: Only include booking with Diagnosis

Note: Lab data excluded

In [119]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP1_DATAPREP = 0

### Import Prep Data Intermediate Data

In [120]:
# Load data
if LOAD_INTERMEDIATE_STEP1_DATAPREP == 1:
    df_diagnosis = read_pickle("step1_booking_diagnosis.pkl")
    df_presciption = read_pickle("step1_booking_prescription.pkl")
    df_symptoms = read_pickle("step1_booking_symptoms.pkl")
    open_file = open("step1_booking_list.pkl", "rb")
    booking_list = pickle.load(open_file)
    open_file.close()
else:
    df_diagnosis = final_df_booking_diagnosis
    df_prescription = final_df_booking_presciption
    df_symptoms = final_df_booking_symptoms
    booking_list = booking_list

In [121]:
print("Booking Diagnosis: ", df_diagnosis.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           9958 non-null   object
 1   diagnosis_code_list  9958 non-null   object
 2   diagnosis_list       9958 non-null   object
 3   chronic_list         9958 non-null   object
 4   count_diagnosis      9958 non-null   object
 5   count_chronic        9958 non-null   object
dtypes: object(6)
memory usage: 544.6+ KB
Booking Diagnosis:  None


In [122]:
print("Booking Prescription:")
print(df_prescription.info())

Booking Prescription:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7606 entries, 0 to 7605
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           7606 non-null   object
 1   medicationID_list    7606 non-null   object
 2   medicationName_list  7606 non-null   object
 3   medQuantity_list     7606 non-null   object
 4   medAndQuan_list      7606 non-null   object
 5   count_medication     7606 non-null   object
 6   count_repeatmed      7606 non-null   object
dtypes: object(7)
memory usage: 475.4+ KB
None


In [123]:
print("Booking Symptoms:")
print(df_symptoms.info())

Booking Symptoms:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8870 entries, 0 to 8869
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        8870 non-null   object
 1   SymptomID_list    8870 non-null   object
 2   SymptomName_list  8870 non-null   object
 3   count_SymID       8870 non-null   object
dtypes: object(4)
memory usage: 346.5+ KB
None


### Import Source Booking Data

In [124]:
# import new booking 
print("Input: ", FILE_BOOKING, "\n") 
df_booking = pd.read_csv(FILE_BOOKING)
df_booking = df_booking[df_booking['booking_id'].isin(booking_list)]
print("Basic Data structure Info (NEW) ", df_booking.info(), "\n") 
print("Shape of data: ", df_booking.shape)

Input:  tbl_Booking_test.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7725 entries, 0 to 7724
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 7725 non-null   object
 1   mem_id                     7725 non-null   object
 2   doctor_id                  7725 non-null   object
 3   consult_waived_reason      7725 non-null   object
 4   is_mc_issued               7725 non-null   bool  
 5   mc_days                    7725 non-null   int64 
 6   is_referral_letter_issued  7725 non-null   bool  
 7   booking_timestamp          7725 non-null   object
dtypes: bool(2), int64(1), object(5)
memory usage: 437.5+ KB
Basic Data structure Info (NEW)  None 

Shape of data:  (7725, 8)


In [125]:
# import old booking 
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_BOOKING_OLD, "\n") 
    df_booking_old = pd.read_csv(FILE_BOOKING_OLD)
    df_booking_old = df_booking_old[df_booking_old['booking_id'].isin(booking_list)]
    print("Basic Data structure Info ", df_booking_old.info(), "\n")
    print("Shape of data: ", df_booking_old.shape)

Input:  tbl_Booking.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2233 entries, 1 to 10219
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 2233 non-null   object
 1   mem_id                     2233 non-null   object
 2   doctor_id                  2233 non-null   object
 3   consult_waived_reason      2233 non-null   object
 4   is_mc_issued               2233 non-null   bool  
 5   mc_days                    2233 non-null   int64 
 6   is_referral_letter_issued  2233 non-null   bool  
 7   booking_timestamp          2233 non-null   object
dtypes: bool(2), int64(1), object(5)
memory usage: 126.5+ KB
Basic Data structure Info  None 

Shape of data:  (2233, 8)


In [126]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1:
    df_booking = pd.concat([df_booking, df_booking_old], axis=0)
    print("Basic Data structure Info ", df_booking.info(), "\n")
    print("Shape of data: ", df_booking.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 10219
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 9958 non-null   object
 1   mem_id                     9958 non-null   object
 2   doctor_id                  9958 non-null   object
 3   consult_waived_reason      9958 non-null   object
 4   is_mc_issued               9958 non-null   bool  
 5   mc_days                    9958 non-null   int64 
 6   is_referral_letter_issued  9958 non-null   bool  
 7   booking_timestamp          9958 non-null   object
dtypes: bool(2), int64(1), object(5)
memory usage: 564.0+ KB
Basic Data structure Info  None 

Shape of data:  (9958, 8)


In [127]:
### Import Source Paitent Data
df_patient = pd.read_csv(FILE_PATIENT)
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   mem_id      6774 non-null   object
 1   gender      6774 non-null   object
 2   birth_year  6774 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 158.9+ KB


### Merge Data

### Combine Booking and Patient Data

In [128]:
combine_book_patient = pd.merge(df_booking, df_patient, how="inner", on=["mem_id"])

In [129]:
combine_book_patient.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 9958 non-null   object
 1   mem_id                     9958 non-null   object
 2   doctor_id                  9958 non-null   object
 3   consult_waived_reason      9958 non-null   object
 4   is_mc_issued               9958 non-null   bool  
 5   mc_days                    9958 non-null   int64 
 6   is_referral_letter_issued  9958 non-null   bool  
 7   booking_timestamp          9958 non-null   object
 8   gender                     9958 non-null   object
 9   birth_year                 9958 non-null   int64 
dtypes: bool(2), int64(2), object(6)
memory usage: 719.6+ KB


In [130]:
print("Unique value count for each column")
count_unique(combine_book_patient)

Unique value count for each column


,name,uniquecount
0,booking_id,9958
1,mem_id,6774
2,doctor_id,30
3,consult_waived_reason,7
4,is_mc_issued,2
5,mc_days,10
6,is_referral_letter_issued,2
7,booking_timestamp,9946
8,gender,3
9,birth_year,62


### Missing Value Checks

In [131]:
print("number of null: ")
combine_book_patient.isnull().sum()

number of null: 


booking_id                   0
mem_id                       0
doctor_id                    0
consult_waived_reason        0
is_mc_issued                 0
mc_days                      0
is_referral_letter_issued    0
booking_timestamp            0
gender                       0
birth_year                   0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Check and remove duplicates

In [132]:
print("number of duplicate to drop: ", combine_book_patient.duplicated().sum())

number of duplicate to drop:  0


In [133]:
# to drop duplicates 
if combine_book_patient.duplicated().sum() > 0:
    combine_book_patient = combine_book_patient.drop_duplicates()

### Combine Base with Diagnoisis

In [134]:
combine_base_diagnosis_inner = pd.merge(combine_book_patient, df_diagnosis, how="inner", on=["booking_id"])

In [135]:
combine_base_diagnosis_inner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 9958 non-null   object
 1   mem_id                     9958 non-null   object
 2   doctor_id                  9958 non-null   object
 3   consult_waived_reason      9958 non-null   object
 4   is_mc_issued               9958 non-null   bool  
 5   mc_days                    9958 non-null   int64 
 6   is_referral_letter_issued  9958 non-null   bool  
 7   booking_timestamp          9958 non-null   object
 8   gender                     9958 non-null   object
 9   birth_year                 9958 non-null   int64 
 10  diagnosis_code_list        9958 non-null   object
 11  diagnosis_list             9958 non-null   object
 12  chronic_list               9958 non-null   object
 13  count_diagnosis            9958 non-null   object
 14  count_ch

### Combine (Base + Diagnosis)Inner + Prescription(Outer)

In [136]:
combine_base_diagInner_presOuter = pd.merge(combine_base_diagnosis_inner, 
                                            df_prescription, how="outer", on=["booking_id"])

In [137]:
combine_base_diagInner_presOuter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 9958 non-null   object
 1   mem_id                     9958 non-null   object
 2   doctor_id                  9958 non-null   object
 3   consult_waived_reason      9958 non-null   object
 4   is_mc_issued               9958 non-null   bool  
 5   mc_days                    9958 non-null   int64 
 6   is_referral_letter_issued  9958 non-null   bool  
 7   booking_timestamp          9958 non-null   object
 8   gender                     9958 non-null   object
 9   birth_year                 9958 non-null   int64 
 10  diagnosis_code_list        9958 non-null   object
 11  diagnosis_list             9958 non-null   object
 12  chronic_list               9958 non-null   object
 13  count_diagnosis            9958 non-null   object
 14  count_ch

### Combine (Base + Diagnosis)Inner + Presc(Outer) + Sym(Outer)

In [138]:
combine_base_diagInner_presOuter_symOuter = pd.merge(combine_base_diagInner_presOuter, df_symptoms, 
                                                     how="outer", on=["booking_id"])

In [139]:
combine_base_diagInner_presOuter_symOuter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 9958 non-null   object
 1   mem_id                     9958 non-null   object
 2   doctor_id                  9958 non-null   object
 3   consult_waived_reason      9958 non-null   object
 4   is_mc_issued               9958 non-null   bool  
 5   mc_days                    9958 non-null   int64 
 6   is_referral_letter_issued  9958 non-null   bool  
 7   booking_timestamp          9958 non-null   object
 8   gender                     9958 non-null   object
 9   birth_year                 9958 non-null   int64 
 10  diagnosis_code_list        9958 non-null   object
 11  diagnosis_list             9958 non-null   object
 12  chronic_list               9958 non-null   object
 13  count_diagnosis            9958 non-null   object
 14  count_ch

In [140]:
final_df = combine_base_diagInner_presOuter_symOuter

### Save Intermediate Output

In [141]:
# Export data 
final_df.to_pickle("step2_merge.pkl")

<span class="burk">COMPLETE STEP 2</span>

## Step3 - Intro Cast & Transpose for Member

To summarize the following categories into columns:
* Chronic
* Diagnosis Code
* Symptoms

### Import Merged Data from Step2

In [142]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP2_DATAPREP = 0

In [143]:
# Load data
if LOAD_INTERMEDIATE_STEP2_DATAPREP == 1:
    df_final_2 = read_pickle("step2_merge.pkl")
else:
    df_final_2 = final_df

In [144]:
print(df_final_2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 9958 non-null   object
 1   mem_id                     9958 non-null   object
 2   doctor_id                  9958 non-null   object
 3   consult_waived_reason      9958 non-null   object
 4   is_mc_issued               9958 non-null   bool  
 5   mc_days                    9958 non-null   int64 
 6   is_referral_letter_issued  9958 non-null   bool  
 7   booking_timestamp          9958 non-null   object
 8   gender                     9958 non-null   object
 9   birth_year                 9958 non-null   int64 
 10  diagnosis_code_list        9958 non-null   object
 11  diagnosis_list             9958 non-null   object
 12  chronic_list               9958 non-null   object
 13  count_diagnosis            9958 non-null   object
 14  count_ch

## Step3a - Cast & Transpose for Chronic

### Check NA for Chronic

In [145]:
df_final_2['chronic_list'] = df_final_2['chronic_list'].fillna("N").apply(list)

### Process Chronic String

In [146]:
df_final_2["chronic_list_str"] = (df_final_2["chronic_list"].apply(lambda x: ",".join(map(str, x))))

In [147]:
#process string
df_final_2['chronic_list_str'] = df_final_2['chronic_list_str'].replace("","NoneChronic")

In [148]:
# To check
df_final_2['chronic_list_str'].head(20)

0                         NoneChronic
1     Vasomotor and allergic rhinitis
2                         NoneChronic
3                         NoneChronic
4                         NoneChronic
5                         NoneChronic
6                         NoneChronic
7                         NoneChronic
8                         NoneChronic
9     Vasomotor and allergic rhinitis
10    Vasomotor and allergic rhinitis
11                        NoneChronic
12                        NoneChronic
13                        NoneChronic
14                        NoneChronic
15                        NoneChronic
16                        NoneChronic
17                        NoneChronic
18                        NoneChronic
19                        NoneChronic
Name: chronic_list_str, dtype: object

### Create Chronic Dict

In [149]:
chronic = df_final_2['chronic_list_str']

In [150]:
#create a dictionary of symptoms with frequency
chronic_dict = {}
for i in chronic:
    li = i.split(",")
    for k in li:
        if k in chronic_dict:
            chronic_dict[k] += 1
        else:
            chronic_dict[k] = 1

print("There are", len(chronic_dict), 
      "unique chronic ! Inclucing all of them as features will be detremental to performance")

There are 11 unique chronic ! Inclucing all of them as features will be detremental to performance


In [151]:
chronic_dict

{'NoneChronic': 8873,
 'Vasomotor and allergic rhinitis': 308,
 'Gout': 90,
 'Atopic dermatitis': 133,
 'Hypertension': 158,
 'Gastro-oesophageal reflux disease': 131,
 'Hyperlipidaemia': 144,
 'Asthma': 84,
 'Migraine': 124,
 'Diabetes': 34,
 'Thyroid': 37}

### Create Chronic Summary DF

In [152]:
chronic_df = pd.DataFrame(columns=chronic_dict.keys(), index=range(len(chronic)))

In [153]:
for chronic_i in (chronic_dict): 
    chronic_df[chronic_i] = chronic.str.contains(chronic_i)

In [154]:
chronic_df = chronic_df.add_prefix("chronic_")

In [155]:
new_df_chronic_summarize = pd.concat([df_final_2["booking_id"], chronic_df], axis=1)

In [156]:
new_df_chronic_summarize.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   booking_id                                 9958 non-null   object
 1   chronic_NoneChronic                        9958 non-null   bool  
 2   chronic_Vasomotor and allergic rhinitis    9958 non-null   bool  
 3   chronic_Gout                               9958 non-null   bool  
 4   chronic_Atopic dermatitis                  9958 non-null   bool  
 5   chronic_Hypertension                       9958 non-null   bool  
 6   chronic_Gastro-oesophageal reflux disease  9958 non-null   bool  
 7   chronic_Hyperlipidaemia                    9958 non-null   bool  
 8   chronic_Asthma                             9958 non-null   bool  
 9   chronic_Migraine                           9958 non-null   bool  
 10  chronic_Diabetes                    

In [157]:
new_df_chronic_summarize.shape

(9958, 12)

In [158]:
# Tocheck
new_df_chronic_summarize["chronic_NoneChronic"].sum()

8873

In [159]:
# Tocheck
new_df_chronic_summarize["chronic_Gout"].sum()

90

In [160]:
new_df_chronic_summarize.head().transpose()

,0,1,2,3,4
booking_id,EA149BC8-BFBC-4703-B34A-E4BB546EC114,3C374977-6526-4612-80FD-D53BE6CDF499,48958FEF-8052-4624-AEAD-792568C230D0,4013B73D-C9A7-45FC-8A4A-817D68EC3E06,6BF25F05-F16A-4ABC-93AA-49218518D40F
chronic_NoneChronic,True,False,True,True,True
chronic_Vasomotor and allergic rhinitis,False,True,False,False,False
chronic_Gout,False,False,False,False,False
chronic_Atopic dermatitis,False,False,False,False,False
chronic_Hypertension,False,False,False,False,False
chronic_Gastro-oesophageal reflux disease,False,False,False,False,False
chronic_Hyperlipidaemia,False,False,False,False,False
chronic_Asthma,False,False,False,False,False
chronic_Migraine,False,False,False,False,False


## Step3b - Cast and Transpose for Diagnosis Code

### Check NA for Diagnosis Code

In [161]:
df_final_2['diagnosis_code_list'] = df_final_2['diagnosis_code_list'].fillna("N").apply(list)

### Process Diagnosis Code String

In [162]:
df_final_2["diagnosis_code_list_str"] = (df_final_2["diagnosis_code_list"].apply(lambda x: ",".join(map(str, x))))

In [163]:
# Remove the dot to reduce the category
diagnosis_code = df_final_2['diagnosis_code_list_str']
diagnosis_code = diagnosis_code.apply(lambda x: re.sub('\.\d+', '', x))

### Create Diagnosis Code Dict

In [164]:
#create a dictionary of symptoms with frequency
diagnosis_code_dict = {}
temp=[]
for i in diagnosis_code:
    li = i.split(",")
    for k in li:
        k = re.sub('\.\d+', '', k)
        if k in temp:
            diagnosis_code_dict[k] += 1
        else:
            diagnosis_code_dict[k] = 1
            temp.append(k)

print("There are", len(diagnosis_code_dict))

There are 315


### (Optional) Set Min Count to include for Features (for model training)

Removed diagnosis that happens less than 10 times, 
to reduce the features, that is equivalent to 0.1% of total number of obervation

In [165]:
# remove min value (in case required to cut)
selected_diag = {key:val for key, val in diagnosis_code_dict.items() if val > MIN_VAL_DIAG}

### Create Diagnosis Code Summary DF

In [166]:
diagnosis_df = pd.DataFrame(columns=selected_diag.keys(), index=range(len(diagnosis_code)))

In [167]:
for diag_i in (selected_diag): 
    diagnosis_df[diag_i] = diagnosis_code.apply(lambda x: str(diag_i) in str(x))

In [168]:
diagnosis_df = diagnosis_df.add_prefix("diag_")

In [169]:
diagnosis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9958 entries, 0 to 9957
Columns: 315 entries, diag_U12 to diag_H26
dtypes: bool(315)
memory usage: 3.0 MB


In [170]:
new_df_diag_summarize = pd.concat([df_final_2["booking_id"], diagnosis_df], axis=1)

In [171]:
new_df_diag_summarize.shape

(9958, 316)

## Step3c - Cast & Transpose for Symptoms

### Check NA for Symptoms 

In [172]:
df_final_2['SymptomName_list'] = df_final_2['SymptomName_list'].fillna("N").apply(list)

### Process Symptoms String

In [173]:
df_final_2["SymptomName_list_str"] = (df_final_2["SymptomName_list"].apply(lambda x: ",".join(map(str, x))))

In [174]:
#process string
symp = df_final_2['SymptomName_list_str']

### Create Symptoms Dict

In [175]:
#create a dictionary of symptoms with frequency
symp_dict = {}
for i in symp:
    li = i.split(",")
    for k in li:
        if k in symp_dict:
            symp_dict[k] += 1
        else:
            symp_dict[k] = 1

print("There are", len(symp_dict), "unique symptoms ! \
      Inclucing all of them as features will be detremental to performance, do check")

There are 21 unique symptoms !       Inclucing all of them as features will be detremental to performance, do check


In [176]:
symp_dict

{'N': 1088,
 'headache': 2355,
 'fever': 1113,
 'giddiness': 2203,
 'others': 3506,
 'medical referral': 318,
 'vomiting': 1245,
 'diarrhoea': 1291,
 'sore throat': 1404,
 'flu': 1376,
 'cough': 1380,
 'cold': 1330,
 'cold sores': 430,
 'rash': 448,
 'chronic medicine top-up': 311,
 'discuss lab results': 335,
 'chronic illness management': 40,
 'skin growth on sole (foot)': 1,
 'ulcer under tongue': 1,
 'vertigo': 1,
 'runny nose': 1}

### (Optional) Set Min Count to include for Features (for modeling)

Removed symptoms that happens less than 10 times, to reduce the features.

In [177]:
# remove min value (in case required to cut)
selected_symptoms = {key:val for key, val in symp_dict.items() if val > MIN_VAL_SYM}

### Create Symptoms Summary DF

In [178]:
symptom_df = pd.DataFrame(columns=selected_symptoms.keys(), index=range(len(symp)))

In [179]:
for symptom in (selected_symptoms): 
    symptom_df[symptom] = symp.str.contains(symptom)

In [180]:
symptom_df = symptom_df.add_prefix("symp_")

In [181]:
new_df_symp_summarize = pd.concat([df_final_2["booking_id"], symptom_df], axis=1)

In [182]:
new_df_symp_summarize.shape

(9958, 22)

## Step4 - Merge Summarized Data

In [183]:
# drop unnec fields
df_final_2.drop(["chronic_list_str", "diagnosis_code_list_str", "SymptomName_list_str"], inplace = True, axis = 1)

In [184]:
combine_expandDiag = pd.merge(df_final_2, new_df_diag_summarize, 
                              how="inner", on=["booking_id"])

In [185]:
combine_expandDiagChronic = pd.merge(combine_expandDiag, new_df_chronic_summarize, 
                                     how="inner", on=["booking_id"])

In [186]:
combine_expandDiagChronicSym = pd.merge(combine_expandDiagChronic, new_df_symp_summarize, 
                                        how="inner", on=["booking_id"])

In [187]:
combine_expandDiagChronicSym.shape

(9958, 371)

### Save intermediate Output

In [188]:
# Export data 
combine_expandDiagChronicSym.to_pickle("step4_merge_summarized.pkl")

<span class="burk">COMPLETE STEP 4</span>

## Step5 - Finalise Booking Data

### Import Step 4 merged data

In [189]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP4_DATAPREP = 0

In [190]:
# Load data
if LOAD_INTERMEDIATE_STEP4_DATAPREP == 1:
    df_final_3 = read_pickle("step4_merge_summarized.pkl")
else:
    df_final_3 = combine_expandDiagChronicSym

### Create new variable - Target

In [191]:
df_final_3["Target_chronic"] = df_final_3["count_chronic"].apply(lambda x: 1 if x>0 else 0)

### Change Category Data Type

In [192]:
# change data types for categorical data
print("Change Data Type to Category")

Change Data Type to Category


In [193]:
cat_list = ["doctor_id", "consult_waived_reason", "gender"]

In [194]:
for col in cat_list:
    df_final_3[col] = df_final_3[col].astype("category")

### Create New Numeric variable - Age

In [195]:
today = date.today()

In [196]:
df_final_3["age_yr"] = today.year - df_final_3["birth_year"]

### Change Category Data Type

In [197]:
num_list = ['mc_days',"age_yr",
 'count_diagnosis',
 'count_chronic',
 'count_medication',
 'count_repeatmed',
 'count_SymID'
 ]

In [198]:
for col in num_list:
    df_final_3[col] = df_final_3[col].astype("float")

### Change Boolean to Binary Dummy

In [199]:
filter_col_diag = [col for col in df_final_3 if col.startswith('diag_')]

In [200]:
filter_col_chronic = [col for col in df_final_3 if col.startswith('chronic_')]

In [201]:
filter_col_symp = [col for col in df_final_3 if col.startswith('symp_')]

In [202]:
fitler_col_other_bool =  ["is_mc_issued", "is_referral_letter_issued", "Target_chronic"]

In [203]:
boolean_list = fitler_col_other_bool + filter_col_diag + filter_col_chronic + filter_col_symp

In [204]:
for col in boolean_list:
    df_final_3[col] = df_final_3[col].apply(lambda x: 1 if x==True else 0)

### Change data types to date time 

In [205]:
# change data types to date time
df_final_3["booking_timestamp"] = pd.to_datetime(df_final_3["booking_timestamp"])

### Create New Variable Consult Hour

In [206]:
# check latest time
df_final_3["consult_hr"] = df_final_3["booking_timestamp"].dt.hour

In [207]:
df_final_3["consult_hr"] = df_final_3["consult_hr"].astype("category")

### Impute Missing Data

In [208]:
miss_list = ["count_medication", "count_repeatmed", "count_SymID"]

In [209]:
for col in miss_list:
    df_final_3[col] = df_final_3[col].fillna(0)

### Impute with "NA"

In [210]:
df_final_3["consult_waived_reason"] = df_final_3["consult_waived_reason"].apply(lambda x: "NA" if x =="-" else x)

In [211]:
df_final_3["consult_waived_reason"].value_counts()

NA                            9464
FOLLOW UP                      214
HEALTH SCREENING REVIEW        192
UNSUITABLE FOR TELECONSULT      51
OTHERS                          22
WHITECOAT FAMILY                14
REFERRAL                         1
Name: consult_waived_reason, dtype: int64

### Export Final at Bookling Level

In [212]:
print("Shape of data: ", df_final_3.shape)

Shape of data:  (9958, 374)


In [213]:
df_final_3.info("deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 374 columns):
 #    Column                                     Dtype         
---   ------                                     -----         
 0    booking_id                                 object        
 1    mem_id                                     object        
 2    doctor_id                                  category      
 3    consult_waived_reason                      category      
 4    is_mc_issued                               int64         
 5    mc_days                                    float64       
 6    is_referral_letter_issued                  int64         
 7    booking_timestamp                          datetime64[ns]
 8    gender                                     category      
 9    birth_year                                 int64         
 10   diagnosis_code_list                        object        
 11   diagnosis_list                             object     

In [214]:
pd.set_option('display.max_rows', None)
df_final_3.head().transpose()

,0,1,2,3,4
booking_id,EA149BC8-BFBC-4703-B34A-E4BB546EC114,3C374977-6526-4612-80FD-D53BE6CDF499,48958FEF-8052-4624-AEAD-792568C230D0,4013B73D-C9A7-45FC-8A4A-817D68EC3E06,6BF25F05-F16A-4ABC-93AA-49218518D40F
mem_id,B3FDAFB4-2BCE-420E-98EF-04E57BEAEDC4,E6A4EC49-02D2-45A2-9F84-053D636A9675,4347A549-532F-4A2B-AD31-0670BB1CA8F3,1C5E3562-9894-41D3-B5C2-077E7D17FE8C,8217E3B4-97B8-4605-B339-08F1E96DF3C8
doctor_id,84599BF3-5C45-4786-9D7C-16CDFA73F1CF,DE28DA35-C28E-49FF-B77C-28E6268F0EF9,DE28DA35-C28E-49FF-B77C-28E6268F0EF9,DE28DA35-C28E-49FF-B77C-28E6268F0EF9,77EFF8BB-D181-4BFB-90A4-75FA494F4BBB
consult_waived_reason,UNSUITABLE FOR TELECONSULT,NA,NA,NA,NA
is_mc_issued,0,1,1,0,1
mc_days,0.0,2.0,1.0,0.0,1.0
is_referral_letter_issued,0,0,0,1,0
booking_timestamp,2022-01-29 13:08:19,2022-01-06 12:25:12,2022-01-13 09:51:08,2022-01-11 20:33:35,2022-01-04 09:03:59
gender,Male,Female,Female,Female,Unknown
birth_year,1993,1976,1993,1969,1989


In [215]:
# Export data 
df_final_3.to_pickle("step5_final_booking_prep.pkl")

In [216]:
if TESTING == 1:
    if MODEL_TYPE_MEMBER ==1:
        df_final_3.to_pickle("booking_test_data_final_forMember.pkl")
    elif MODEL_TYPE_MEMBER ==0:
        df_final_3.to_pickle("booking_test_data_final_forBooking.pkl")
    else:
        print("Pls check the modeling purpose for the output testing.")
else: 
    df_final_3.to_pickle("booking_train_data_final.pkl")

<span class="burk">COMPLETE STEP 5</span>

## Step6 - Data Aggregate by Member

### Import Step 5 Finalised Booking Data

In [217]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP5_BOOK = 0

In [218]:
# Load data
if LOAD_INTERMEDIATE_STEP5_BOOK == 1:
    df_booking = read_pickle("step5_final_booking_prep.pkl")
else:
    df_booking = df_final_3

### Data Aggregation: New Features

#### Get tot numer of booking per member

In [219]:
mem_agg1 = df_booking.groupby(["mem_id"])["booking_id"].count()
mem_agg1 = mem_agg1.reset_index()

In [220]:
mem_agg1.rename(columns = {'mem_id' : 'mem_id', 'booking_id' : 'Mem_CountBooking'}, inplace = True)

In [221]:
count_unique(mem_agg1)

,name,uniquecount
0,mem_id,6774
1,Mem_CountBooking,17


In [222]:
mem_agg1["Mem_CountBooking"].value_counts()

1     5362
2      811
3      214
4      124
5       74
6       73
7       45
8       23
9       18
10      11
11       8
13       4
20       2
15       2
19       1
16       1
12       1
Name: Mem_CountBooking, dtype: int64

#### Get mean for aggregation

In [223]:
df_booking.info("deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9958 entries, 0 to 9957
Data columns (total 374 columns):
 #    Column                                     Dtype         
---   ------                                     -----         
 0    booking_id                                 object        
 1    mem_id                                     object        
 2    doctor_id                                  category      
 3    consult_waived_reason                      category      
 4    is_mc_issued                               int64         
 5    mc_days                                    float64       
 6    is_referral_letter_issued                  int64         
 7    booking_timestamp                          datetime64[ns]
 8    gender                                     category      
 9    birth_year                                 int64         
 10   diagnosis_code_list                        object        
 11   diagnosis_list                             object     

In [224]:
select_list = [
     'mc_days',
     'count_diagnosis',
     'count_chronic',
     'count_medication',
     'count_repeatmed',
     'count_SymID',
     'age_yr']

In [225]:
mem_agg2 = df_booking.groupby(["mem_id"])[select_list].mean()
mem_agg2 = mem_agg2.reset_index()

In [226]:
mem_agg2 = mem_agg2.rename(columns={col: 'AVE_' + col
                        for col in mem_agg2.columns if col not in ['mem_id']})

In [227]:
new_col2 = mem_agg2.columns.tolist()

In [228]:
# Round off value to int
for col in new_col2[1:]:
    mem_agg2[col]  =  mem_agg2[col].apply(lambda x: int(x))

In [229]:
mem_agg2.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
AVE_mc_days,6774.0,1.268379,1.539772,0.0,0.0,1.0,1.0,7.0
AVE_count_diagnosis,6774.0,1.160614,0.422190,1.0,1.0,1.0,1.0,5.0
AVE_count_chronic,6774.0,0.087983,0.317221,0.0,0.0,0.0,0.0,3.0
AVE_count_medication,6774.0,1.875996,1.584746,0.0,1.0,2.0,3.0,10.0
AVE_count_repeatmed,6774.0,0.709477,0.460815,0.0,0.0,1.0,1.0,3.0
AVE_count_SymID,6774.0,1.916150,1.471937,0.0,1.0,2.0,2.0,10.0
AVE_age_yr,6774.0,34.418660,8.711145,16.0,28.0,33.0,39.0,85.0


#### Get max for aggregation

In [230]:
mem_agg3 = df_booking.groupby(["mem_id"])[select_list].max()
mem_agg3 = mem_agg3.reset_index()

In [231]:
mem_agg3 = mem_agg3.rename(columns={col: 'MAX_' + col
                        for col in mem_agg3.columns if col not in ['mem_id']})

In [232]:
new_col3 = mem_agg3.columns.tolist()

In [233]:
# Round off value to int
for col in new_col3[1:]:
    mem_agg3[col]  =  mem_agg3[col].apply(lambda x: int(x))

In [234]:
mem_agg3.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
MAX_mc_days,6774.0,1.490995,1.726328,0.0,0.0,1.0,2.0,14.0
MAX_count_diagnosis,6774.0,1.252879,0.526826,1.0,1.0,1.0,1.0,6.0
MAX_count_chronic,6774.0,0.131680,0.392710,0.0,0.0,0.0,0.0,3.0
MAX_count_medication,6774.0,2.169029,1.731900,0.0,1.0,2.0,3.0,13.0
MAX_count_repeatmed,6774.0,0.797609,0.412688,0.0,1.0,1.0,1.0,3.0
MAX_count_SymID,6774.0,2.155300,1.600542,0.0,1.0,2.0,3.0,10.0
MAX_age_yr,6774.0,34.418660,8.711145,16.0,28.0,33.0,39.0,85.0


#### Get min for aggregation

In [235]:
mem_agg4 = df_booking.groupby(["mem_id"])[select_list].min()
mem_agg4 = mem_agg4.reset_index()

In [236]:
mem_agg4 = mem_agg4.rename(columns={col: 'MIN_' + col
                        for col in mem_agg4.columns if col not in ['mem_id']})

In [237]:
new_col4 = mem_agg4.columns.tolist()

In [238]:
# Round off value to int
for col in new_col4[1:]:
    mem_agg4[col]  =  mem_agg4[col].apply(lambda x: int(x))

In [239]:
mem_agg4.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
MIN_mc_days,6774.0,1.183643,1.537155,0.0,0.0,1.0,1.0,7.0
MIN_count_diagnosis,6774.0,1.154414,0.415001,1.0,1.0,1.0,1.0,5.0
MIN_count_chronic,6774.0,0.085179,0.312128,0.0,0.0,0.0,0.0,3.0
MIN_count_medication,6774.0,1.728521,1.622945,0.0,0.0,1.0,3.0,10.0
MIN_count_repeatmed,6774.0,0.709330,0.460882,0.0,0.0,1.0,1.0,3.0
MIN_count_SymID,6774.0,1.819457,1.480629,0.0,1.0,1.0,2.0,10.0
MIN_age_yr,6774.0,34.418660,8.711145,16.0,28.0,33.0,39.0,85.0


In [240]:
mem_agg4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   mem_id                6774 non-null   object
 1   MIN_mc_days           6774 non-null   int64 
 2   MIN_count_diagnosis   6774 non-null   int64 
 3   MIN_count_chronic     6774 non-null   int64 
 4   MIN_count_medication  6774 non-null   int64 
 5   MIN_count_repeatmed   6774 non-null   int64 
 6   MIN_count_SymID       6774 non-null   int64 
 7   MIN_age_yr            6774 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 423.5+ KB


#### Check Age

In [241]:
dff_max_mean_age = mem_agg3["MAX_age_yr"] - mem_agg2["AVE_age_yr"]

In [242]:
sum(dff_max_mean_age>0)

0

In [243]:
dff_max_min_age = mem_agg3["MAX_age_yr"] - mem_agg4["MIN_age_yr"]

In [244]:
sum(dff_max_min_age>0)

0

In [245]:
# Conclude: Age is consistet, can drop Min and Max
mem_agg3.drop(["MAX_age_yr"], axis=1, inplace=True)
mem_agg4.drop(["MIN_age_yr"], axis=1, inplace=True)


In [246]:
mem_agg2.rename(columns = {'AVE_age_yr' : 'age_yr'}, inplace = True)

In [247]:
mem_agg2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   mem_id                6774 non-null   object
 1   AVE_mc_days           6774 non-null   int64 
 2   AVE_count_diagnosis   6774 non-null   int64 
 3   AVE_count_chronic     6774 non-null   int64 
 4   AVE_count_medication  6774 non-null   int64 
 5   AVE_count_repeatmed   6774 non-null   int64 
 6   AVE_count_SymID       6774 non-null   int64 
 7   age_yr                6774 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 423.5+ KB


#### Get count for aggregation for Waived_Reason

In [248]:
mem_agg5 = df_booking.groupby(["mem_id",'consult_waived_reason'])["booking_id"].count()
mem_agg5 = mem_agg5.reset_index()

In [249]:
mem_agg5.rename(columns = {'mem_id' : 'mem_id', "consult_waived_reason": "consult_waived_reason", 
                           'booking_id' : 'Reason_CountBooking'}, inplace = True)

In [250]:
mem_agg5.head()

,mem_id,consult_waived_reason,Reason_CountBooking
0,00059B0F-87F5-4D64-A5C8-441457913282,NA,1
1,00059B0F-87F5-4D64-A5C8-441457913282,FOLLOW UP,0
2,00059B0F-87F5-4D64-A5C8-441457913282,HEALTH SCREENING REVIEW,0
3,00059B0F-87F5-4D64-A5C8-441457913282,OTHERS,0
4,00059B0F-87F5-4D64-A5C8-441457913282,REFERRAL,0


In [251]:
mem_agg5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47418 entries, 0 to 47417
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   mem_id                 47418 non-null  object  
 1   consult_waived_reason  47418 non-null  category
 2   Reason_CountBooking    47418 non-null  int64   
dtypes: category(1), int64(1), object(1)
memory usage: 787.7+ KB


In [252]:
mem_agg5_2 = pd.pivot_table(mem_agg5, index="mem_id", columns='consult_waived_reason', 
                            values='Reason_CountBooking')

In [253]:
mem_agg5_2 = mem_agg5_2.reset_index()

In [254]:
mem_agg5_2.head(10)

consult_waived_reason,mem_id,NA,FOLLOW UP,HEALTH SCREENING REVIEW,OTHERS,REFERRAL,UNSUITABLE FOR TELECONSULT,WHITECOAT FAMILY
0,00059B0F-87F5-4D64-A5C8-441457913282,1,0,0,0,0,0,0
1,000789AE-E161-4FDE-B9B0-9377AB383A1B,1,0,0,0,0,0,0
2,0008AEAF-0767-46A2-830F-C98A9B88FF34,1,0,0,0,0,0,0
3,000C3E29-2C23-4EB0-A1FB-837EC7EA9E59,1,0,0,0,0,0,0
4,001C4B02-1827-4F58-85E8-AB985B27496A,1,0,0,0,0,0,0
5,0023F228-597C-451A-9311-F200CC05CC31,1,0,0,0,0,0,0
6,0026B107-D521-4150-94E1-C488C9F6D97C,8,0,0,0,0,0,0
7,0031C903-CFC8-4F3E-8E25-82AC003720A1,1,0,0,0,0,0,0
8,00324C3A-C33F-4FFA-BA9E-47534B69D3A1,1,0,0,0,0,0,0
9,00362462-1425-40E5-8CDA-10A01493B36B,2,0,0,0,0,0,0


In [255]:
mem_agg5_2 = mem_agg5_2.rename(columns={col: 'WaviedCount_' + col
                        for col in mem_agg5_2.columns if col not in ['mem_id']})

In [256]:
mem_agg5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   mem_id                                  6774 non-null   object
 1   WaviedCount_NA                          6774 non-null   int64 
 2   WaviedCount_FOLLOW UP                   6774 non-null   int64 
 3   WaviedCount_HEALTH SCREENING REVIEW     6774 non-null   int64 
 4   WaviedCount_OTHERS                      6774 non-null   int64 
 5   WaviedCount_REFERRAL                    6774 non-null   int64 
 6   WaviedCount_UNSUITABLE FOR TELECONSULT  6774 non-null   int64 
 7   WaviedCount_WHITECOAT FAMILY            6774 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 423.5+ KB


In [257]:
mem_agg5_2.columns = mem_agg5_2.columns.str.replace(' ','_')

In [258]:
mem_agg5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   mem_id                                  6774 non-null   object
 1   WaviedCount_NA                          6774 non-null   int64 
 2   WaviedCount_FOLLOW_UP                   6774 non-null   int64 
 3   WaviedCount_HEALTH_SCREENING_REVIEW     6774 non-null   int64 
 4   WaviedCount_OTHERS                      6774 non-null   int64 
 5   WaviedCount_REFERRAL                    6774 non-null   int64 
 6   WaviedCount_UNSUITABLE_FOR_TELECONSULT  6774 non-null   int64 
 7   WaviedCount_WHITECOAT_FAMILY            6774 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 423.5+ KB


In [259]:
mem_agg5_2.columns

Index(['mem_id', 'WaviedCount_NA', 'WaviedCount_FOLLOW_UP',
       'WaviedCount_HEALTH_SCREENING_REVIEW', 'WaviedCount_OTHERS',
       'WaviedCount_REFERRAL', 'WaviedCount_UNSUITABLE_FOR_TELECONSULT',
       'WaviedCount_WHITECOAT_FAMILY'],
      dtype='object', name='consult_waived_reason')

#### Get count for aggregation for ConsultHr

In [260]:
mem_agg6 = df_booking.groupby(["mem_id",'consult_hr'])["booking_id"].count()
mem_agg6 = mem_agg6.reset_index()

In [261]:
mem_agg6.rename(columns = {'mem_id' : 'mem_id', "consult_hr": "consult_hr", 
                           'booking_id' : 'ConsultHr_CountBooking'}, inplace = True)

In [262]:
mem_agg6_2 = pd.pivot_table(mem_agg6, index="mem_id", columns='consult_hr', 
                            values='ConsultHr_CountBooking')

In [263]:
mem_agg6_2 = mem_agg6_2.reset_index()

In [264]:
mem_agg6_2.head()

consult_hr,mem_id,0,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,00059B0F-87F5-4D64-A5C8-441457913282,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,000789AE-E161-4FDE-B9B0-9377AB383A1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0008AEAF-0767-46A2-830F-C98A9B88FF34,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,000C3E29-2C23-4EB0-A1FB-837EC7EA9E59,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,001C4B02-1827-4F58-85E8-AB985B27496A,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [265]:
mem_agg6_2 = mem_agg6_2.rename(columns={col: 'ConsultHr_' + str(col)
                        for col in mem_agg6_2.columns if col not in ['mem_id']})

In [266]:
mem_agg6_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mem_id        6774 non-null   object
 1   ConsultHr_0   6774 non-null   int64 
 2   ConsultHr_7   6774 non-null   int64 
 3   ConsultHr_8   6774 non-null   int64 
 4   ConsultHr_9   6774 non-null   int64 
 5   ConsultHr_10  6774 non-null   int64 
 6   ConsultHr_11  6774 non-null   int64 
 7   ConsultHr_12  6774 non-null   int64 
 8   ConsultHr_13  6774 non-null   int64 
 9   ConsultHr_14  6774 non-null   int64 
 10  ConsultHr_15  6774 non-null   int64 
 11  ConsultHr_16  6774 non-null   int64 
 12  ConsultHr_17  6774 non-null   int64 
 13  ConsultHr_18  6774 non-null   int64 
 14  ConsultHr_19  6774 non-null   int64 
 15  ConsultHr_20  6774 non-null   int64 
 16  ConsultHr_21  6774 non-null   int64 
 17  ConsultHr_22  6774 non-null   int64 
 18  ConsultHr_23  6774 non-null   int64 
dtypes: int

In [267]:
mem_agg6_2.columns

Index(['mem_id', 'ConsultHr_0', 'ConsultHr_7', 'ConsultHr_8', 'ConsultHr_9',
       'ConsultHr_10', 'ConsultHr_11', 'ConsultHr_12', 'ConsultHr_13',
       'ConsultHr_14', 'ConsultHr_15', 'ConsultHr_16', 'ConsultHr_17',
       'ConsultHr_18', 'ConsultHr_19', 'ConsultHr_20', 'ConsultHr_21',
       'ConsultHr_22', 'ConsultHr_23'],
      dtype='object', name='consult_hr')

#### Get sum for aggregation for Dummy Variable

In [268]:
filter_col_diag = [col for col in df_booking if col.startswith('diag_')]

In [269]:
filter_col_chronic = [col for col in df_booking if col.startswith('chronic_')]

In [270]:
filter_col_symp = [col for col in df_booking if col.startswith('symp_')]

In [271]:
fitler_col_other_bool =  ["is_mc_issued", "is_referral_letter_issued", "Target_chronic"]

In [272]:
select_list_dummy = fitler_col_other_bool + filter_col_diag + filter_col_chronic + filter_col_symp

In [273]:
# must change to numeric to do sum
for col in select_list_dummy:
    df_booking[col] = df_booking[col].astype("int")

In [274]:
mem_agg7 = df_booking.groupby(["mem_id"])[select_list_dummy].sum()
mem_agg7 = mem_agg7.reset_index()

In [275]:
mem_agg7 = mem_agg7.rename(columns={col: 'CountVisit_' + col
                        for col in mem_agg7.columns if col not in ['mem_id']})

In [276]:
mem_agg7.columns = mem_agg7.columns.str.replace(' ','_')

Reminder: The count of ChronicVisit, e.g. CountVisit_Target_chronic, refers to visits with Chronic status marked, it idoes not related to Chronic diagnostic the visit.

#### Get first dates related aggregation

In [277]:
book_firstdate = df_booking.groupby(["mem_id"])["booking_timestamp"].min()
book_firstdate = book_firstdate.reset_index()

In [278]:
book_firstdate.rename(columns = {'mem_id' : 'mem_id', 'booking_timestamp' : 'FirstBookDate'}, inplace = True)

In [279]:
book_firstdate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   mem_id         6774 non-null   object        
 1   FirstBookDate  6774 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 106.0+ KB


In [280]:
book_firstdate.head()

,mem_id,FirstBookDate
0,00059B0F-87F5-4D64-A5C8-441457913282,2022-01-10 11:55:10
1,000789AE-E161-4FDE-B9B0-9377AB383A1B,2022-01-06 22:50:03
2,0008AEAF-0767-46A2-830F-C98A9B88FF34,2022-01-29 18:42:48
3,000C3E29-2C23-4EB0-A1FB-837EC7EA9E59,2022-01-03 14:07:34
4,001C4B02-1827-4F58-85E8-AB985B27496A,2022-01-15 09:32:59


#### Get last dates related aggregation

In [281]:
book_lastdate = df_booking.groupby(["mem_id"])["booking_timestamp"].max()
book_lastdate = book_lastdate.reset_index()

In [282]:
book_lastdate.rename(columns = {'mem_id' : 'mem_id', 'booking_timestamp' : 'LastBookDate'}, inplace = True)

#### Get Mode for aggregation Consult Waived Reason

In [283]:
mem_agg8 = df_booking.groupby(["mem_id"])["consult_waived_reason"].apply(lambda x: x.value_counts().index[0])
mem_agg8 = mem_agg8.reset_index() 

In [284]:
mem_agg8.rename(columns = {'mem_id' : 'mem_id', 'consult_waived_reason' : 'Mode_consult_waived_reason'}, inplace = True)

In [285]:
mem_agg8["Mode_consult_waived_reason"].value_counts()

NA                            6547
HEALTH SCREENING REVIEW        166
UNSUITABLE FOR TELECONSULT      29
FOLLOW UP                       17
OTHERS                           8
WHITECOAT FAMILY                 7
Name: Mode_consult_waived_reason, dtype: int64

#### Get Mode for aggregation Consultation Hour

In [286]:
mem_agg9 = df_booking.groupby(["mem_id"])["consult_hr"].apply(lambda x: x.value_counts().index[0])
mem_agg9 = mem_agg9.reset_index()

In [287]:
mem_agg9.rename(columns = {'mem_id' : 'mem_id', 'consult_hr' : 'Mode_consult_hr'}, inplace = True)

#### Get gender for member

In [288]:
mem_gender = df_booking.groupby(["mem_id"])["gender"].apply(lambda x: x.value_counts().index[0])
mem_gender = mem_gender.reset_index() 

In [289]:
mem_gender["gender"].value_counts()

Female     4130
Male       2159
Unknown     485
Name: gender, dtype: int64

### Merge Aggregation

* mem_agg1 : Mem_CountBooking
* mem_agg2 : 'mem_id', 'AVE_mc_days', 'AVE_count_diagnosis', 'AVE_count_chronic',
       'AVE_count_medication', 'AVE_count_repeatmed', 'AVE_count_SymID',
       'age_yr'
* mem_agg3 : 'mem_id', 'MIN_mc_days', 'MIN_count_diagnosis', 'MIN_count_chronic',
       'MIN_count_medication', 'MIN_count_repeatmed', 'MIN_count_SymID'
* mem_agg4 : 'mem_id', 'MIN_mc_days', 'MIN_count_diagnosis', 'MIN_count_chronic',
       'MIN_count_medication', 'MIN_count_repeatmed', 'MIN_count_SymID'
* mem_agg5_2 : 'WaviedCount_NA','WaviedCount_FOLLOW_UP','WaviedCount_HEALTH_SCREENING_REVIEW',
        'WaviedCount_OTHERS','WaviedCount_REFERRAL','WaviedCount_UNSUITABLE_FOR_TELECONSULT','WaviedCount_WHITECOAT_FAMILY'
* mem_agg6_2: 'ConsultHr_0','ConsultHr_8','ConsultHr_9','ConsultHr_10','ConsultHr_11',
         'ConsultHr_12','ConsultHr_13','ConsultHr_14','ConsultHr_15','ConsultHr_16',
         'ConsultHr_17','ConsultHr_18','ConsultHr_19','ConsultHr_20','ConsultHr_21',
         'ConsultHr_22','ConsultHr_23'     
* mem_agg7: 'CountVisit_is_mc_issued',
         'CountVisit_is_referral_letter_issued',
         'CountVisit_diag_H57',
         'CountVisit_diag_M23',
         'CountVisit_diag_T88',
         'CountVisit_diag_L20',
         'CountVisit_diag_R51',
         'CountVisit_diag_N94',
         'CountVisit_diag_G44',
         'CountVisit_diag_M79',
         'CountVisit_diag_J02',
         'CountVisit_diag_J06',
         'CountVisit_diag_G43',
         'CountVisit_diag_R21',
         'CountVisit_diag_M62',
         'CountVisit_diag_A09',
         'CountVisit_diag_J30',
         'CountVisit_diag_L30',
         'CountVisit_diag_T78',
         'CountVisit_diag_U07',
         'CountVisit_diag_K29',
         'CountVisit_diag_B35',
         'CountVisit_diag_K30',
         'CountVisit_diag_H81',
         'CountVisit_diag_N39',
         'CountVisit_diag_G47',
         'CountVisit_diag_Z76',
         'CountVisit_diag_K59',
         'CountVisit_diag_H10',
         'CountVisit_diag_R42',
         'CountVisit_diag_K21',
         'CountVisit_diag_S90',
         'CountVisit_diag_R50',
         'CountVisit_diag_U12',
         'CountVisit_diag_N92',
         'CountVisit_diag_M54',
         'CountVisit_diag_S60',
         'CountVisit_diag_I10',
         'CountVisit_diag_L03',
         'CountVisit_diag_K64',
         'CountVisit_diag_L98',
         'CountVisit_diag_B37',
         'CountVisit_diag_L73',
         'CountVisit_diag_H00',
         'CountVisit_diag_N76',
         'CountVisit_diag_J01',
         'CountVisit_diag_J45',
         'CountVisit_diag_E78',
         'CountVisit_diag_L02',
         'CountVisit_diag_R52',
         'CountVisit_diag_K12',
         'CountVisit_diag_K58',
         'CountVisit_diag_K07',
         'CountVisit_diag_L70',
         'CountVisit_diag_Z71',
         'CountVisit_diag_S63',
         'CountVisit_diag_M25',
         'CountVisit_diag_K05',
         'CountVisit_diag_E03',
         'CountVisit_diag_L50',
         'CountVisit_diag_W57',
         'CountVisit_diag_H93',
         'CountVisit_diag_H01',
         'CountVisit_diag_E11',
         'CountVisit_diag_S13',
         'CountVisit_diag_F41',
         'CountVisit_diag_R53',
         'CountVisit_diag_O91',
         'CountVisit_diag_H02',
         'CountVisit_diag_R11',
         'CountVisit_diag_S93',
         'CountVisit_diag_L24',
         'CountVisit_diag_J39',
         'CountVisit_diag_J32',
         'CountVisit_diag_B00',
         'CountVisit_diag_L64',
         'CountVisit_diag_Y42',
         'CountVisit_diag_R10',
         'CountVisit_diag_M10',
         'CountVisit_diag_M75',
         'CountVisit_diag_Z02',
         'CountVisit_diag_J03',
         'CountVisit_diag_R07',
         'CountVisit_diag_K13',
         'CountVisit_diag_Z30',
         'CountVisit_diag_P39',
         'CountVisit_diag_R22',
         'CountVisit_diag_H60',
         'CountVisit_diag_B02',
         'CountVisit_diag_L29',
         'CountVisit_diag_E05',
         'CountVisit_diag_N64',
         'CountVisit_diag_R05',
         'CountVisit_diag_L60',
         'CountVisit_diag_S80',
         'CountVisit_diag_M65',
         'CountVisit_diag_J22',
         'CountVisit_diag_O21',
         'CountVisit_diag_M72',
         'CountVisit_diag_L23',
         'CountVisit_diag_K52',
         'CountVisit_chronic_NoneChronic',
         'CountVisit_chronic_Atopic dermatitis',
         'CountVisit_chronic_Migraine',
         'CountVisit_chronic_Vasomotor and allergic rhinitis',
         'CountVisit_chronic_Gastro-oesophageal reflux disease',
         'CountVisit_chronic_Hypertension',
         'CountVisit_chronic_Asthma',
         'CountVisit_chronic_Hyperlipidaemia',
         'CountVisit_chronic_Thyroid',
         'CountVisit_chronic_Diabetes',
         'CountVisit_chronic_Gout',
         'CountVisit_symp_others',
         'CountVisit_symp_medical referral',
         'CountVisit_symp_headache',
         'CountVisit_symp_giddiness',
         'CountVisit_symp_fever',
         'CountVisit_symp_N',
         'CountVisit_symp_cold sores',
         'CountVisit_symp_rash',
         'CountVisit_symp_sore throat',
         'CountVisit_symp_flu',
         'CountVisit_symp_cough',
         'CountVisit_symp_cold',
         'CountVisit_symp_vomiting',
         'CountVisit_symp_diarrhoea',
         'CountVisit_symp_chronic medicine top-up',
         'CountVisit_symp_chronic illness management',
         'CountVisit_symp_discuss lab results',
         'CountVisit_Target_chronic'
* book_firstdate: 'FirstBookDate'
* book_lastdate: 'LastBookDate'
* mem_agg8: 'Mode_consult_waived_reason'
* mem_agg9: "Mode_consult_hr"
* mem_gender: gender
 

In [290]:
# initiate dataframe
new_df = pd.DataFrame() 

In [291]:
new_df = pd.merge(mem_gender,mem_agg1, how="inner", on=["mem_id"])

In [292]:
new_df = pd.merge(new_df, mem_agg2, how="inner", on=["mem_id"])

In [293]:
new_df = pd.merge(new_df, mem_agg3, how="inner", on=["mem_id"])

In [294]:
new_df = pd.merge(new_df, mem_agg4, how="inner", on=["mem_id"])

In [295]:
new_df = pd.merge(new_df, mem_agg5_2, how="inner", on=["mem_id"])

In [296]:
new_df = pd.merge(new_df, mem_agg6_2, how="inner", on=["mem_id"])

In [297]:
new_df = pd.merge(new_df, mem_agg7, how="inner", on=["mem_id"])

In [298]:
new_df = pd.merge(new_df, mem_agg8, how="inner", on=["mem_id"])

In [299]:
new_df = pd.merge(new_df, mem_agg9, how="inner", on=["mem_id"])

In [300]:
new_df = pd.merge(new_df, book_firstdate, how="inner", on=["mem_id"])

In [301]:
new_df = pd.merge(new_df, book_lastdate, how="inner", on=["mem_id"])

In [302]:
new_df.shape

(6774, 402)

### Add Feature - Diff between Last and First Date

In [303]:
new_df["DiffDays_Vist"] = new_df["LastBookDate"]-new_df["FirstBookDate"]

In [304]:
new_df["DiffDays_Vist"]  = new_df["DiffDays_Vist"].apply(lambda x: abs(x.days))

### Add Feature - Average Visit Freq

In [305]:
new_df["Ave_VisitGapDays"] = new_df["DiffDays_Vist"] / new_df["Mem_CountBooking"]

In [306]:
new_df["Ave_VisitGapDays"]  = new_df["Ave_VisitGapDays"].apply(lambda x: int(x))

### Create Targets

In [307]:
new_df["Target_Chronic"] = new_df["CountVisit_Target_chronic"].apply(lambda x : 1 if x >0 else 0)

In [308]:
new_df["Target_Dermatitis"] = new_df["CountVisit_chronic_Atopic_dermatitis"].apply(lambda x : 1 if x >0 else 0)

In [309]:
new_df["Target_Migraine"] = new_df["CountVisit_chronic_Migraine"].apply(lambda x : 1 if x >0 else 0)

In [310]:
new_df["Target_Allergic"] = new_df["CountVisit_chronic_Vasomotor_and_allergic_rhinitis"].apply(lambda x : 1 if x >0 else 0)

In [311]:
new_df["Target_Gastro"] = new_df["CountVisit_chronic_Gastro-oesophageal_reflux_disease"].apply(lambda x : 1 if x >0 else 0)

In [312]:
new_df["Target_Hypertension"] = new_df["CountVisit_chronic_Hypertension"].apply(lambda x : 1 if x >0 else 0)

In [313]:
new_df["Target_Asthma"] = new_df["CountVisit_chronic_Asthma"].apply(lambda x : 1 if x >0 else 0)

In [314]:
new_df["Target_Hyperlipidaemia"] = new_df["CountVisit_chronic_Hyperlipidaemia"].apply(lambda x : 1 if x >0 else 0)

In [315]:
new_df["Target_Thyroid"] = new_df["CountVisit_chronic_Thyroid"].apply(lambda x : 1 if x >0 else 0)

In [316]:
new_df["Target_Diabetes"] = new_df["CountVisit_chronic_Diabetes"].apply(lambda x : 1 if x >0 else 0)

In [317]:
new_df["Target_Gout"] = new_df["CountVisit_chronic_Gout"].apply(lambda x : 1 if x >0 else 0)

### Export Prep Data at Member level

In [318]:
# Export data 
new_df.to_pickle("step6_member_prep.pkl")

## Step7 -  Define Group Targets

### Import Step 6 Aggregated Member Data

In [319]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP6_MEM = 0

In [320]:
# Load data
if LOAD_INTERMEDIATE_STEP6_MEM == 1:
    new_df = read_pickle("step6_member_prep.pkl")
else:
    new_df = new_df

### Define 3 High - Diabetes, Hyperlipidaemia, Hypertension

In [321]:
new_df["temp_3High"] = new_df["Target_Hyperlipidaemia"] + new_df["Target_Diabetes"] + new_df["Target_Hypertension"]
new_df["Target_3High"] = new_df["temp_3High"].apply(lambda x: 1 if x>0 else 0)

In [322]:
new_df.Target_Diabetes.value_counts()

0    6750
1      24
Name: Target_Diabetes, dtype: int64

In [323]:
new_df.Target_Hyperlipidaemia.value_counts()

0    6683
1      91
Name: Target_Hyperlipidaemia, dtype: int64

In [324]:
new_df.Target_Hypertension.value_counts()

0    6682
1      92
Name: Target_Hypertension, dtype: int64

In [325]:
new_df.Target_3High.value_counts()

0    6618
1     156
Name: Target_3High, dtype: int64

In [326]:
new_df = new_df.drop(columns=["temp_3High"])

### Define Allergic_Dermatitis_Asthma

In [327]:
# Target for allergic related - Atopic_dermatitis, Vasomotor_and_allergic_rhinitic, Asthma
new_df["temp_Allergic_Dermatitis_Asthma"] = new_df["Target_Dermatitis"] + new_df["Target_Allergic"] + new_df["Target_Asthma"]
new_df["Target_Allergic_Dermatitis_Asthma"] = new_df["temp_Allergic_Dermatitis_Asthma"].apply(lambda x: 1 if x>0 else 0)

In [328]:
new_df.Target_Dermatitis.value_counts()

0    6657
1     117
Name: Target_Dermatitis, dtype: int64

In [329]:
new_df.Target_Allergic.value_counts()

0    6525
1     249
Name: Target_Allergic, dtype: int64

In [330]:
new_df.Target_Asthma.value_counts()

0    6711
1      63
Name: Target_Asthma, dtype: int64

In [331]:
new_df["temp_Allergic_Dermatitis_Asthma"].value_counts()

0    6383
1     356
2      32
3       3
Name: temp_Allergic_Dermatitis_Asthma, dtype: int64

In [332]:
new_df["Target_Allergic_Dermatitis_Asthma"].value_counts()

0    6383
1     391
Name: Target_Allergic_Dermatitis_Asthma, dtype: int64

In [333]:
new_df = new_df.drop(columns=["temp_Allergic_Dermatitis_Asthma"])

### Define Migraine_Gastro_Thyroid_Gou

In [334]:
new_df["temp_Migraine_Gastro_Thyroid_Gout"] = new_df["Target_Migraine"] + new_df["Target_Gastro"] + new_df["Target_Thyroid"] + new_df["Target_Gout"]
new_df["Target_Migraine_Gastro_Thyroid_Gout"] = new_df["temp_Migraine_Gastro_Thyroid_Gout"].apply(lambda x: 1 if x>0 else 0)

In [335]:
new_df.Target_Migraine.value_counts()

0    6685
1      89
Name: Target_Migraine, dtype: int64

In [336]:
new_df.Target_Gastro.value_counts()

0    6663
1     111
Name: Target_Gastro, dtype: int64

In [337]:
new_df.Target_Thyroid.value_counts()

0    6753
1      21
Name: Target_Thyroid, dtype: int64

In [338]:
new_df.Target_Gout.value_counts()

0    6700
1      74
Name: Target_Gout, dtype: int64

In [339]:
new_df["Target_Migraine_Gastro_Thyroid_Gout"].value_counts()

0    6489
1     285
Name: Target_Migraine_Gastro_Thyroid_Gout, dtype: int64

In [340]:
new_df = new_df.drop(columns=["temp_Migraine_Gastro_Thyroid_Gout"])

### Group Consultancy Hour

Group Consultancy Hour based on PCA and MCA analysis

In [341]:
new_df["ConsultHr_8to10"] = new_df["ConsultHr_8"] + new_df["ConsultHr_9"] + new_df["ConsultHr_10"]

In [342]:
new_df["ConsultHr_11to13"] = new_df["ConsultHr_11"] + new_df["ConsultHr_12"] + new_df["ConsultHr_13"]

In [343]:
new_df["ConsultHr_14to17"] = new_df["ConsultHr_14"] + new_df["ConsultHr_15"] + new_df["ConsultHr_16"] + new_df["ConsultHr_17"]

In [344]:
new_df["ConsultHr_18to21"] = new_df["ConsultHr_18"] + new_df["ConsultHr_19"] + new_df["ConsultHr_20"] + new_df["ConsultHr_21"]

In [345]:
new_df["ConsultHr_22to23"] = new_df["ConsultHr_22"] + new_df["ConsultHr_23"] 

### Export Prep Member Data with Chronic Grouping

In [346]:
# Export data 
new_df.to_pickle("step7_member_grp_prep.pkl")

## Step8 - Finalise Member Data

As some of the syptoms are very highly correalated. PCA is performed on the highly corrected variables using JMP software to generate out the rules for deriving the principal components. 

### Import Step 7 Finalised Member Data with Chronic Grouping

In [347]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP7_MEM_GP = 0

In [348]:
# Load data
if LOAD_INTERMEDIATE_STEP7_MEM_GP == 1:
    df_mem = read_pickle("step7_member_grp_prep.pkl")
else:
    df_mem = new_df

### Create New Features for Principal Components for Symptoms

new columns = ['Prin1_sym_common',
       'Prin2_sym_rash', 'Prin3_sym_headache', 'Prin4_sym_diarrhea']

#### Principal Component1 - Common 

Formula from JMP: 
Prin1_sym_common = 0.126922208999099 * :CountVisit_symp_headache + 0.134938007927841 *
:CountVisit_symp_giddiness + 0.457991577692383 * :CountVisit_symp_cold_sores
+0.448844403126012 * :CountVisit_symp_rash + 0.954160558888485 *
:CountVisit_symp_sore_throat + 0.960039882968821 * :CountVisit_symp_flu
+0.966269306714024 * :CountVisit_symp_cough + 0.744793202425646 *
:CountVisit_symp_cold + 0.111674590804738 * :CountVisit_symp_vomiting
+0.105420843334157 * :CountVisit_symp_diarrhoea + (-1.08200575730171)

In [349]:
df_mem["Prin1_sym_common"] = 0.126922208999099 * df_mem["CountVisit_symp_headache"] + \
                             0.134938007927841 * df_mem["CountVisit_symp_giddiness"] + \
                             0.457991577692383 * df_mem["CountVisit_symp_cold_sores"] + \
                             0.448844403126012 * df_mem["CountVisit_symp_rash"] + \
                             0.954160558888485 * df_mem["CountVisit_symp_sore_throat"] + \
                             0.960039882968821 * df_mem["CountVisit_symp_flu"] + \
                             0.966269306714024 * df_mem["CountVisit_symp_cough"] + \
                             0.744793202425646 * df_mem["CountVisit_symp_cold"] + \
                             0.111674590804738 * df_mem["CountVisit_symp_vomiting"] + \
                             0.105420843334157 * df_mem["CountVisit_symp_diarrhoea"] - 1.08200575730171

#### Principal Component - Rash

Formula from JMP: 
Prin2_sym_rash = 0.15012952272156 * :CountVisit_symp_headache + 0.157099331295396 *
:CountVisit_symp_giddiness + 1.4198008192801 * :CountVisit_symp_cold_sores
+1.39667010805915 * :CountVisit_symp_rash + -0.509538387575815 *
:CountVisit_symp_sore_throat + -0.51973978746157 * :CountVisit_symp_flu +
-0.518711617511583 * :CountVisit_symp_cough + 0.286064554630894 *
:CountVisit_symp_cold + 0.281708989416286 * :CountVisit_symp_vomiting
+0.268761673670889 * :CountVisit_symp_diarrhoea + (-0.399412355479641)

In [350]:
df_mem["Prin2_sym_rash"] = 0.15012952272156 * df_mem["CountVisit_symp_headache"] + \
                            0.157099331295396 * df_mem["CountVisit_symp_giddiness"] + \
                            1.4198008192801 * df_mem["CountVisit_symp_cold_sores"] + \
                            1.39667010805915 * df_mem["CountVisit_symp_rash"] - \
                            0.509538387575815 * df_mem["CountVisit_symp_sore_throat"] - \
                            0.51973978746157 * df_mem["CountVisit_symp_flu"] - \
                            0.518711617511583 * df_mem["CountVisit_symp_cough"] + \
                            0.286064554630894 * df_mem["CountVisit_symp_cold"] + \
                            0.281708989416286 * df_mem["CountVisit_symp_vomiting"] + \
                            0.268761673670889 * df_mem["CountVisit_symp_diarrhoea"] -0.399412355479641

#### Principal Component - Headache 

Formula from JMP: 
Prin3_sym_headache = 0.53393749284155 * :CountVisit_symp_headache + 0.549256657956352 *
:CountVisit_symp_giddiness + -0.540351073738283 * :CountVisit_symp_cold_sores +
-0.532878377771104 * :CountVisit_symp_rash + -0.0177007121608483 *
:CountVisit_symp_sore_throat + -0.0186518204119694 * :CountVisit_symp_flu +
-0.0183140476024677 * :CountVisit_symp_cough + -0.233303688008869 *
:CountVisit_symp_cold + 0.786678691868385 * :CountVisit_symp_vomiting
+0.762367704391997 * :CountVisit_symp_diarrhoea + (-0.711589799772594)

In [351]:
df_mem["Prin3_sym_headache"] = 0.53393749284155 * df_mem["CountVisit_symp_headache"] + \
                                0.549256657956352 * df_mem["CountVisit_symp_giddiness"] - \
                                0.540351073738283 * df_mem["CountVisit_symp_cold_sores"] - \
                                0.532878377771104 * df_mem["CountVisit_symp_rash"] - \
                                0.0177007121608483 * df_mem["CountVisit_symp_sore_throat"] - \
                                0.0186518204119694 * df_mem["CountVisit_symp_flu"] - \
                                0.0183140476024677 * df_mem["CountVisit_symp_cough"] - \
                                0.233303688008869 * df_mem["CountVisit_symp_cold"] + \
                                0.786678691868385 * df_mem["CountVisit_symp_vomiting"] + \
                                0.762367704391997 * df_mem["CountVisit_symp_diarrhoea"] -0.711589799772594

#### Principal Component - Diarrhea

Formula from JMP: 
Prin4_sym_diarrhea = -0.571563615572338 * :CountVisit_symp_headache + -0.587138492269664 *
:CountVisit_symp_giddiness + -0.0255056350216958 * :CountVisit_symp_cold_sores +
-0.0200870916998447 * :CountVisit_symp_rash + 0.0708239066134195 *
:CountVisit_symp_sore_throat + 0.0595575562025494 * :CountVisit_symp_flu
+0.0650590997464691 * :CountVisit_symp_cough + 0.0197435600586072 *
:CountVisit_symp_cold + 0.835757956281788 * :CountVisit_symp_vomiting
+0.807263138560611 * :CountVisit_symp_diarrhoea + 0.0422774813710458

In [352]:
df_mem["Prin4_sym_diarrhea"] = -0.571563615572338 * df_mem["CountVisit_symp_headache"] - \
                                0.587138492269664 * df_mem["CountVisit_symp_giddiness"] - \
                                0.0255056350216958 * df_mem["CountVisit_symp_cold_sores"] - \
                                0.0200870916998447 * df_mem["CountVisit_symp_rash"] + \
                                0.0708239066134195 * df_mem["CountVisit_symp_sore_throat"] + \
                                0.0595575562025494 * df_mem["CountVisit_symp_flu"] + \
                                0.0650590997464691 * df_mem["CountVisit_symp_cough"] + \
                                0.0197435600586072 * df_mem["CountVisit_symp_cold"] + \
                                0.835757956281788 * df_mem["CountVisit_symp_vomiting"] + \
                                0.807263138560611 * df_mem["CountVisit_symp_diarrhoea"] + 0.0422774813710458

### Export Final Member Data for modeling or Testing

In [353]:
df_mem.info("deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6774 entries, 0 to 6773
Data columns (total 427 columns):
 #    Column                                                Dtype         
---   ------                                                -----         
 0    mem_id                                                object        
 1    gender                                                object        
 2    Mem_CountBooking                                      int64         
 3    AVE_mc_days                                           int64         
 4    AVE_count_diagnosis                                   int64         
 5    AVE_count_chronic                                     int64         
 6    AVE_count_medication                                  int64         
 7    AVE_count_repeatmed                                   int64         
 8    AVE_count_SymID                                       int64         
 9    age_yr                                                int64  

In [354]:
df_mem.shape

(6774, 427)

In [355]:
# Export data 
df_mem.to_pickle("step8_final_member.pkl")

In [356]:
if TESTING == 1:
    df_mem.to_pickle("member_test_data_final.pkl")
else: 
    df_mem.to_pickle("member_train_data_final.pkl")

@end